In [132]:
# Import our dependencies
# !pip install keras_tuner
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import keras_tuner as kt
import datetime
import shutil
import keras
import ast
import re
import winsound
import json

# Use for creating a beeping sound when processing is finished
duration = 1000  # milliseconds
freq = 440  # Hz


#  Import and read the charity_data.csv.
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 60)
application_df = pd.read_csv("./DataFiles/HR_Analytics.csv")
final_results_all = []
application_df.head()

,EmpID,Age,AgeGroup,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,SalarySlab,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,RM297,18,18-25,Yes,Travel_Rarely,230,Research & Development,3,3,Life Sciences,1,405,3,Male,54,3,1,Laboratory Technician,3,Single,1420,Upto 5k,25233,1,Y,No,13,3,3,80,0,0,2,3,0,0,0,0.0
1,RM302,18,18-25,No,Travel_Rarely,812,Sales,10,3,Medical,1,411,4,Female,69,2,1,Sales Representative,3,Single,1200,Upto 5k,9724,1,Y,No,12,3,1,80,0,0,2,3,0,0,0,0.0
2,RM458,18,18-25,Yes,Travel_Frequently,1306,Sales,5,3,Marketing,1,614,2,Male,69,3,1,Sales Representative,2,Single,1878,Upto 5k,8059,1,Y,Yes,14,3,4,80,0,0,3,3,0,0,0,0.0
3,RM728,18,18-25,No,Non-Travel,287,Research & Development,5,2,Life Sciences,1,1012,2,Male,73,3,1,Research Scientist,4,Single,1051,Upto 5k,13493,1,Y,No,15,3,4,80,0,0,2,3,0,0,0,0.0
4,RM829,18,18-25,Yes,Non-Travel,247,Research & Development,8,1,Medical,1,1156,3,Male,80,3,1,Laboratory Technician,3,Single,1904,Upto 5k,13556,1,Y,No,12,3,4,80,0,0,0,3,0,0,0,0.0


In [133]:
application_df.columns

Index(['EmpID', 'Age', 'AgeGroup', 'Attrition', 'BusinessTravel', 'DailyRate',
       'Department', 'DistanceFromHome', 'Education', 'EducationField',
       'EmployeeCount', 'EmployeeNumber', 'EnvironmentSatisfaction', 'Gender',
       'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobRole',
       'JobSatisfaction', 'MaritalStatus', 'MonthlyIncome', 'SalarySlab',
       'MonthlyRate', 'NumCompaniesWorked', 'Over18', 'OverTime',
       'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction',
       'StandardHours', 'StockOptionLevel', 'TotalWorkingYears',
       'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany',
       'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager'],
      dtype='object')

In [134]:
application_df[['EmpID', 'EmployeeNumber']].sort_values('EmployeeNumber')

,EmpID,EmployeeNumber
1013,RM001,1
1281,RM002,2
804,RM003,4
520,RM004,5
163,RM005,7
...,...,...
210,RM1468,2064
1304,RM1469,2065
1305,RM1469,2065
655,RM1470,2068


In [135]:
application_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1480 entries, 0 to 1479
Data columns (total 38 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   EmpID                     1480 non-null   object 
 1   Age                       1480 non-null   int64  
 2   AgeGroup                  1480 non-null   object 
 3   Attrition                 1480 non-null   object 
 4   BusinessTravel            1480 non-null   object 
 5   DailyRate                 1480 non-null   int64  
 6   Department                1480 non-null   object 
 7   DistanceFromHome          1480 non-null   int64  
 8   Education                 1480 non-null   int64  
 9   EducationField            1480 non-null   object 
 10  EmployeeCount             1480 non-null   int64  
 11  EmployeeNumber            1480 non-null   int64  
 12  EnvironmentSatisfaction   1480 non-null   int64  
 13  Gender                    1480 non-null   object 
 14  HourlyRa

In [136]:
application_df.corr()

C:\Users\T852\AppData\Local\Temp\ipykernel_45704\1123807899.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  application_df.corr()


,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,MonthlyRate,NumCompaniesWorked,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
Age,1.000000,0.011584,-0.000329,0.205689,NaN,-0.011181,0.011617,0.022704,0.026804,0.509758,-0.005269,0.497486,0.029529,0.300739,0.000864,0.000146,0.053632,NaN,0.038545,0.680837,-0.016819,-0.021704,0.310550,0.213330,0.216755,0.209987
DailyRate,0.011584,1.000000,-0.007134,-0.016750,NaN,-0.054996,0.019176,0.021349,0.043964,0.001940,0.034562,0.005963,-0.028730,0.038991,0.021772,-0.001379,0.012214,NaN,0.042097,0.015036,0.004297,-0.037683,-0.033737,0.010452,-0.032439,-0.022281
DistanceFromHome,-0.000329,-0.007134,1.000000,0.019661,NaN,0.038137,-0.013633,0.027899,0.006427,0.007360,-0.004244,-0.015817,0.025782,-0.028873,0.036682,0.024221,0.002762,NaN,0.045153,0.007802,-0.035882,-0.030581,0.010111,0.018593,0.011817,0.010849
Education,0.205689,-0.016750,0.019661,1.000000,NaN,0.038020,-0.026831,0.019446,0.042908,0.097107,-0.012740,0.092182,-0.026779,0.125433,-0.009984,-0.023366,-0.005762,NaN,0.016717,0.145258,-0.026476,0.008806,0.065983,0.057557,0.051507,0.068713
EmployeeCount,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EmployeeNumber,-0.011181,-0.054996,0.038137,0.038020,NaN,1.000000,0.021359,0.030896,-0.007231,-0.016891,-0.051062,-0.013928,0.008998,-0.005340,-0.012501,-0.021808,-0.074316,NaN,0.057548,-0.013731,0.021525,0.009202,-0.010756,-0.008735,-0.011731,-0.009781
EnvironmentSatisfaction,0.011617,0.019176,-0.013633,-0.026831,NaN,0.021359,1.000000,-0.053826,-0.013254,0.000260,-0.010201,-0.006913,0.040251,0.013240,-0.032042,-0.031625,0.009256,NaN,0.002250,-0.002472,-0.016531,0.024867,-0.000318,0.017658,0.013731,0.003072
HourlyRate,0.022704,0.021349,0.027899,0.019446,NaN,0.030896,-0.053826,1.000000,0.046327,-0.028585,-0.070670,-0.015981,-0.018883,0.021169,-0.008834,0.000750,0.000071,NaN,0.049756,-0.004128,-0.010846,-0.004835,-0.018848,-0.023756,-0.026131,-0.003166
JobInvolvement,0.026804,0.043964,0.006427,0.042908,NaN,-0.007231,-0.013254,0.046327,1.000000,-0.013770,-0.018036,-0.017496,-0.019553,0.015537,-0.014514,-0.025382,0.032878,NaN,0.022028,-0.007114,-0.018256,-0.008954,-0.022974,0.005862,-0.025929,0.037990
JobLevel,0.509758,0.001940,0.007360,0.097107,NaN,-0.016891,0.000260,-0.028585,-0.013770,1.000000,-0.001440,0.949835,0.039256,0.142472,-0.036669,-0.021588,0.018063,NaN,0.015803,0.781880,-0.017370,0.038027,0.535210,0.390189,0.355518,0.371987


In [137]:
#application_df.corr().to_csv('./DataFiles/hr_analysis_correlation.csv', index=False)

# Define Functions

In [138]:
# Delete old dirctory used by keras_tuner.Hyperband before a new keras_tuner.Hyperbad is made
def delete_directory_and_its_content(directory_name):

    # Use shutil.rmtree() to recursively delete directories and subdirectories
    try:
        shutil.rmtree(directory_name)
        print(f"Directory '{directory_name}' has been deleted successfully.")
    except OSError as e:
        print(f"Error: {e}")

In [139]:
# Creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()


    # number of layers including input layer but excluding output layer
    num_layers = hp.Int('num_layers', 1, 6)

    activation = hp.Choice('activation_layer_input', ['relu', 'tanh', 'leaky_relu'])

    #  Allow kerastuner to decide which activation function to use in hidden layers
    if activation == 'relu':
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('neurons_layer_input', min_value=X.shape[1], max_value=round(X.shape[1] * 2), step=5), activation='relu', input_dim=X.shape[1]))
    elif activation == 'tanh':
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('neurons_layer_input', min_value=X.shape[1], max_value=round(X.shape[1] * 2), step=5), activation='tanh', input_dim=X.shape[1]))
    elif activation == 'leaky_relu':
        activation = tf.keras.layers.LeakyReLU(alpha=0.01)
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('neurons_layer_input', min_value=X.shape[1], max_value=round(X.shape[1] * 2), step=5), activation=tf.keras.layers.LeakyReLU(alpha=0.01), input_dim=X.shape[1]))


    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(1, num_layers):

        activation = hp.Choice(f'activation_layer_{i}', ['relu', 'leaky_relu'])

        #  Allow kerastuner to decide which activation function to use in hidden layers
        if activation == 'leaky_relu':
            activation = tf.keras.layers.LeakyReLU(alpha=0.01)
        nn_model.add(tf.keras.layers.Dense(units=hp.Int(f'neurons_layer_{i}',
            min_value=2,
            max_value=round(X.shape[1] * 2),
            step=1),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # hp.Choice('random_seed2', [str(np.random.get_state())])
    # Compile the model
    lr = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')

    nn_model.compile(loss="binary_crossentropy", optimizer=keras.optimizers.Adam(learning_rate=lr), metrics = ["accuracy"])#optimizer='adam', metrics=["accuracy"])

    return nn_model

In [140]:
# Function to otherize values from multiple columns
def otherizations(df, list_of_columns_to_otherize, list_of_list_of_values_to_replace):
    for index in range(len(list_of_columns_to_otherize)):
        df = otherize_column(df, list_of_columns_to_otherize[index], list_of_list_of_values_to_replace[index])

    return df

In [141]:

# Function to change value of column to "other" (typically used for rare values)
def otherize_column(df, column, list_of_values_to_replace):
    df_copy = df.copy()


    # Replace in dataframe
    for app in list_of_values_to_replace:
        df_copy[column] = df_copy[column].replace(app,"Other")

    # Check to make sure binning was successful
    # print(df_copy[column].value_counts())

    return df_copy

In [142]:
# Remove outliers from a given column when a value's z-score for that column is greater than a given limit
def remove_outliers(df, threshold, columns_to_remove_outliers_from):
    print(columns_to_remove_outliers_from)
    print(type(columns_to_remove_outliers_from))
    # Calculate Z-scores only for specified columns
    z_scores = np.abs(stats.zscore(df[columns_to_remove_outliers_from]))

    # Create a mask for outliers
    outliers = (z_scores > threshold)

    # Create a DataFrame with outliers for further inspection if needed
    outlier_rows = df[df.index.isin(df[columns_to_remove_outliers_from].index[outliers.any(axis=1)])]

    # Remove rows with outliers in the specified columns
    df_cleaned = df[~df.index.isin(df[columns_to_remove_outliers_from].index[outliers.any(axis=1)])]

    print("Outlier rows:")
    print(outlier_rows.index)
    
    return outlier_rows, df_cleaned

In [143]:
# Remove outliers using the 1.5 InterQuartile Range Rule as the cuttoff values
def remove_outliers_1_5_IQR_Rule(df):
    for column in df.columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
    
        # Filter out rows outside of the IQR for this column
        df = df[(df[column] >= Q1 - 1.5 * IQR) & (df[column] <= Q3 + 1.5 * IQR)]

    return df

In [144]:
# Extract X and y from the given dataframe
def extract_X_y(df, y_column_name):
    # Split our preprocessed data into our features and target arrays
    # print(a2pplication_df.columns)
    X = df.drop(columns=[y_column_name])
    y = df[y_column_name]
    return (X, y)

In [145]:
# Create a specific model instead of rely on keras_tuner.hyperband
def create_specific_model(list_of_activations, num_layers, list_of_number_of_nodes_hidden_layer, input_dim, output_activation, filepath_to_model_weights):

    nn = tf.keras.models.Sequential()

    for layer_number in range(0, num_layers):
        if list_of_activations[layer_number] == 'leaky_relu':
            activation = tf.keras.layers.LeakyReLU(alpha=0.01)
        else:
            activation = list_of_activations[layer_number]
        if layer_number == 0:
            # Input layer which requires input_dim
            nn.add(tf.keras.layers.Dense(units = list_of_number_of_nodes_hidden_layer[layer_number], activation = activation, input_dim = input_dim))
        else:
            nn.add(tf.keras.layers.Dense(units = list_of_number_of_nodes_hidden_layer[layer_number], activation = activation))


    # Output layer
    nn.add(tf.keras.layers.Dense(units = 1, activation = output_activation))
    
    if (filepath_to_model_weights != ''):
        nn.load_weights(filepath_to_model_weights)

    # Check the structure of the model
    print(nn.summary())
    return nn

In [146]:
# Find and return X_train_scaled and X_test_scaled
def scale_it(X_train, X_test):
    # Create a StandardScaler instances
    scaler = StandardScaler()

    # Fit the StandardScaler
    X_scaler = scaler.fit(X_train)

    # Scale the data
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)

    return X_train_scaled, X_test_scaled

In [147]:
# Expose Random Seed of Tensorflow
class RandomSeedLogger(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        # Access the TensorFlow random seed and log it
        random_seed = tf.random.get_seed()
        print(f"Random Seed: {random_seed}")

# Create an instance of the custom callback
random_seed_logger = RandomSeedLogger()

In [148]:
# Otherize not value explicitly but by knowing the ordinal popularity of the value (e.g. '11' for 11th most common value)
def otherizer2(df, column, starting_from_least_column):
    to_replace = [df[column].value_counts().index[starting_from_least_column:]]
    # print(f'len(df[column].value_counts().index): {len(df[column].value_counts().index.tolist())}')
    print(f'len(to_replace): {len(to_replace[0].tolist())}')
    # print(f'to_replace: {to_replace}')
    # Replace in dataframe
    for cls in to_replace:
        df[column] = df[column].replace(cls,"Other")

    # print(f'len(df[column].value_counts().index): {len(df[column].value_counts().index.tolist())}')
    return df

In [149]:
# Optimization by otherizing across a range of values
def optimization2(optimization_title, df, columns, start, finish, step, reset_df):
    if reset_df:
        df = application_df.drop(columns=['EIN', 'NAME'])
    for column in columns:
        if len(df[column].value_counts()) > 5:
            print(f'start: {start}')
            for starting_from_least_column in range(start, finish, step):
                if reset_df:
                    df = application_df.drop(columns=['EIN', 'NAME'])

                print(f'start: {start}')
                print(f'starting_from_least_column: {starting_from_least_column}')
                print(f'len(df[column].value_counts().index): {len(df[column].value_counts().index.tolist())}')
                df = otherizer2(df, column, starting_from_least_column)
                print(f'len(df[column].value_counts().index): {len(df[column].value_counts().index.tolist())}')
                run_hyperparameters(optimization_title + f': otherize columns from {starting_from_least_column}', df, 8, 9, '', 20)

In [150]:
# Return the last n lines of a file
def read_last_n_lines(file_name, n):
    
    with open(file_name, 'r') as file:
        lines = file.readlines()
        return lines[-n:]

In [151]:
# Inserts a newline character every interval (to breakup long strings)
def insert_newlines(string, interval):
    return '\n'.join(string[i:i+interval] for i in range(0, len(string), interval))

In [152]:
def get_test_data_without_removing_outliers(df, df_used_to_make_model, remove_columns):
    df_new = df.copy()

    # These columns to be removed BEFORE pd.get_dummies()
    if remove_columns in df_new.columns:
        df_new_removed_column = df_new.drop(columns=[remove_columns])
    else:
        df_new_removed_column = df_new


    df_new_removed_column = pd.get_dummies(df_new_removed_column)

    # In order to use the same model the number of columns has to match AFTER creating columns from
    # using the pd.get_dummies() method but before finding the X and y
    absent_columns = list(set(df_new_removed_column.columns) - set(df_used_to_make_model.columns))
    if len(absent_columns) > 0:
        df_new_removed_column = df_new_removed_column.drop(columns=absent_columns)
    
    



    # Split our preprocessed data into our features and target array
    X, y = extract_X_y(df_new_removed_column, 'Attrition')




    file_path_shape = "./DataFiles/shape_details.txt"

    # Open the file in append mode
    with open(file_path_shape, "a") as file:
        file.write('in get_test_data_without_removing_outliers(): X and y\n' + str(X.shape) + '\n' + str(y.shape) + '\n')


    # Split the preprocessed data into a training and testing dataset
    X_train, X_test, y_train, y_test = train_test_split(X,
                                                        y,
                                                        random_state=4,
                                                        stratify=y)
    
    
    # Open the file in append mode
    with open(file_path_shape, "a") as file:
        file.write('in get_test_data_without_removing_outliers(): X_train and X_test\n' + str(X_train.shape) + '\n' + str(X_test.shape) + '\n')



    # Standardize X_train and X_test
    X_train_scaled, X_test_scaled = scale_it(X_train, X_test)
    
    return X_test_scaled, y_test

In [153]:
def create_nice_string(dict_stuffs):
    formatted_output = ""
    for key, value in dict_stuffs.items():
        if isinstance(value, dict):  # For nested dictionaries, convert them to string without newlines
            value_str = json.dumps(value)
        else:
            value_str = str(value)
        formatted_output += f"'{key}': {value_str}\n"
    return '{' + formatted_output + '}\n'

In [189]:
# Create hyperband models for multiple z-score thresholds
def run_hyperparameters(optimization_title, df_with_preprocessing, df_without_some_preprocessing_for_test_data, columns_to_remove_outliers_from, begin_threshold, end_threshold, column_to_remove, max_epochs, restrict_columns_to_interquartile, use_equal_number_outliers_as_signal_to_bypass_z_score_threshold_analysis, run_model_against_X_test_scaled_with_outliers):
    # X must be global for create_model to receive X
    global X#, y, final_results_all, X_train, X_test, y_train, y_test, X_train_scaled, X_test_scaled

    if column_to_remove in df_with_preprocessing.columns:
        df_new_removed_column = df_with_preprocessing.drop(columns=[column_to_remove])
    else:
        df_new_removed_column = df_with_preprocessing



    # Set option to display all rows
    pd.set_option('display.max_rows', None)

    df_new_removed_column = pd.get_dummies(df_new_removed_column)
    
    for z_threshold in range(begin_threshold, end_threshold):

        np.random.seed(42)
        tf.random.set_seed(42)
        number_of_rows_before_outliers_removed = df_new_removed_column.shape[0]

        if restrict_columns_to_interquartile:
            no_outliers_df = remove_outliers_1_5_IQR_Rule(df_new_removed_column)
        else:
            outlier_rows, no_outliers_df = remove_outliers(df_new_removed_column, z_threshold, columns_to_remove_outliers_from)

        number_of_outlier_rows = number_of_rows_before_outliers_removed - no_outliers_df.shape[0]
        

        file_path = "./DataFiles/optimization_results.txt"
        past_results_records = read_last_n_lines(file_path, 8)
        prev_number_of_outlier_rows = re.search("'number of outlier rows': (\d+)", past_results_records[1]).group(1)
        print(prev_number_of_outlier_rows)
        if use_equal_number_outliers_as_signal_to_bypass_z_score_threshold_analysis:
            if int(prev_number_of_outlier_rows) == number_of_outlier_rows:
                # Skip to the next threshold
                continue
        #     pass
        # print('number of columns', len(df_new_removed_column.columns))
        # return

        # Split our preprocessed data into our features and target array
        X, y = extract_X_y(no_outliers_df, 'Attrition')

        

        file_path_shape = "./DataFiles/shape_details.txt"

        # Open the file in append mode
        with open(file_path_shape, "a") as file:
            file.write('in hyperparameters(): X and y\n' + str(X.shape) + '\n' + str(y.shape) + '\n')

        # Split the preprocessed data into a training and testing dataset
        X_train, X_test, y_train, y_test = train_test_split(X,
                                                            y,
                                                            random_state=3,
                                                            stratify=y)

        # Standardize X_train and X_test
        X_train_scaled, X_test_scaled = scale_it(X_train, X_test)


        # Open the file in append mode
        with open(file_path_shape, "a") as file:
            file.write('in hyperparameters(): X_train_scaled and X_test_scaled\n' + str(X_train_scaled.shape) + '\n' + str(X_test_scaled.shape) + '\n')

        delete_directory_and_its_content('./untitled_project/')


        
        tuner = kt.Hyperband(
            create_model,
            objective="val_accuracy",
            max_epochs=max_epochs,
            hyperband_iterations=2,
            seed=1)
        
        # Run the kerastuner search for best hyperparameters
        tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))
        first_model = tuner.get_best_models(num_models=1)[0]


        # First evaluate WITHOUT outliers in test data
        model_loss, model_accuracy_WITHOUT_outliers_in_test_data = first_model.evaluate(X_test_scaled,y_test,verbose=2)


        timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        first_model.save(f'./DataFiles/model_{timestamp}_{float(model_accuracy_WITHOUT_outliers_in_test_data*100):.2f}.h5')

        dict_results = {}
        dict_results['model_accuracy_WITHOUT_outliers_in_test_data'] = model_accuracy_WITHOUT_outliers_in_test_data


        # Get test data WITH outliers still present but note that b/c the same model from above will be used the columns will have to match
        # in the df that is sent
        X_test_scaled_WITH_outliers, y_test_WITH_outliers = get_test_data_without_removing_outliers(df_without_some_preprocessing_for_test_data, no_outliers_df, column_to_remove)


        # Open the file in append mode
        with open(file_path_shape, "a") as file:
            file.write('in hyperparameters(): X_test_scaled_WITH_outliers and y_test_WITH_outliers\n' + str(X_test_scaled_WITH_outliers.shape) + '\n' + str(y_test_WITH_outliers.shape) + '\n')

        dict_results['timestamp'] = timestamp
        dict_results['z_threshold'] = z_threshold
        dict_results['columns to remove outliers from'] = columns_to_remove_outliers_from
        if run_model_against_X_test_scaled_with_outliers:
            model_loss, model_accuracy_WITH_outliers_in_test_data = first_model.evaluate(X_test_scaled_WITH_outliers,y_test_WITH_outliers,verbose=2)
            dict_results['model_accuracy_WITH_outliers_in_test_data'] = model_accuracy_WITH_outliers_in_test_data
        else:
            dict_results['model_accuracy_WITH_outliers_in_test_data'] = 'did not run with outliers included'
        
        dict_results['number of columns'] = len(df_new_removed_column.columns)
        dict_results['param'] = tuner.get_best_hyperparameters(1)[0].values
        dict_results['columns'] = df_new_removed_column.columns
        dict_results['removed column'] = column_to_remove
        dict_results['restrict columns to 1.5 * IQR'] = restrict_columns_to_interquartile
        dict_results['number of outlier rows'] = number_of_outlier_rows
        dict_results['number of rows'] = no_outliers_df.shape[0]
        dict_results['max_epoch'] = max_epochs
        dict_results['hyperband seed'] = 1
        dict_results['np.rand.seed and tf.random.set_seed'] = 42
        dict_results['Optimization Title'] = optimization_title
        final_results_all.append(dict_results)

        file_path = "./DataFiles/optimization_results.txt"

        # Open the file in append mode
        with open(file_path, "a") as file:
            
            file.write(create_nice_string(dict_results))
    


    # Set option to display all rows
    pd.set_option('display.max_rows', 60)
    print(final_results_all[(-1 * (end_threshold - begin_threshold)):])

In [155]:
# Bucket all 'column' values that had an average Attrition value between min and max inclusively
def bucket_by_Attrition_Average(df, min, max, column, replacement_value):
    df1 = df.copy()

    print(f"Number of unique values in '{column}' are: {len(df1[column].value_counts())}")
    print(df1[column].value_counts())

    groupby_df = df1[[column, 'Attrition']].groupby(column).mean().sort_values('Attrition')

    bucket = groupby_df[(groupby_df['Attrition'] >= min) & (groupby_df['Attrition'] <= max)].index.tolist()    
    print(f"\nBucket '{replacement_value}' includes the values: {bucket}\n")
    # Replace in dataframe
    for cls in bucket:
        df1[column] = df1[column].replace(cls, replacement_value)

    # Check to make sure binning was successful
    print(f"Number of unique values in '{column}' are: {len(df1[column].value_counts())}")
    print(df1[column].value_counts())

    return df1

# Preprocessing: Remove Duplicate Rows

In [156]:
print(f'dataframe shape before duplicate removal: {application_df.shape}')
duplicates_removed_df = application_df.drop_duplicates()
print(f'dataframe shape after duplicate removal: {duplicates_removed_df.shape}')

dataframe shape before duplicate removal: (1480, 38)
dataframe shape after duplicate removal: (1473, 38)


# Preprocessing: Remove Duplicate EmpID Rows

In [157]:

print(f'dataframe shape after duplicate removal: {duplicates_removed_df.shape}')

# Create a dictionary with column names and their unique value counts
unique_counts = {col:duplicates_removed_df[col].nunique() for col in duplicates_removed_df.columns}

# Convert the dictionary to a DataFrame
unique_counts_df = pd.DataFrame(list(unique_counts.items()), columns=['Column Name', 'Unique Values'])

# Display the new DataFrame
print(unique_counts_df)

dataframe shape after duplicate removal: (1473, 38)
                 Column Name  Unique Values
0                      EmpID           1470
1                        Age             43
2                   AgeGroup              5
3                  Attrition              2
4             BusinessTravel              4
5                  DailyRate            886
6                 Department              3
7           DistanceFromHome             29
8                  Education              5
9             EducationField              6
10             EmployeeCount              1
11            EmployeeNumber           1470
12   EnvironmentSatisfaction              4
13                    Gender              2
14                HourlyRate             71
15            JobInvolvement              4
16                  JobLevel              5
17                   JobRole              9
18           JobSatisfaction              4
19             MaritalStatus              3
20             MonthlyIn

### Note
Because the shape shows there are 1473 rows and there are only 1470 unique EmpIDs, that means there are 3 rows with duplicate EmpIDs. And after a search in excel the following show the duplicate EmpIDs. Only one row for each duplicate EmpID will be kept, the other removed. The criteria for removal will be the row with the smaller value in the 'YearsWithCurrManager' column will be removed (presuming that the row with more years is more up-to-date than the row with fewer years).

In [158]:
duplicates_removed_df.loc[duplicates_removed_df['EmpID'] == 'RM1465']

,EmpID,Age,AgeGroup,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,SalarySlab,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
161,RM1465,26,26-35,No,Travel_Rarely,1167,Sales,5,3,Other,1,2060,4,Female,30,2,1,Sales Representative,3,Single,2966,Upto 5k,21378,0,Y,No,18,3,4,80,0,5,2,3,4,2,0,0.0
162,RM1465,26,26-35,No,Travel_Rarely,1167,Sales,5,3,Other,1,2060,4,Female,30,2,1,Sales Representative,3,Single,2966,Upto 5k,21378,0,Y,No,18,3,4,80,0,5,2,3,4,2,0,5.0


In [159]:
duplicates_removed_df.loc[duplicates_removed_df['EmpID'] == 'RM1466']

,EmpID,Age,AgeGroup,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,SalarySlab,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
802,RM1466,36,36-45,No,Travel_Frequently,884,Research & Development,23,2,Medical,1,2061,3,Male,41,4,2,Laboratory Technician,4,Married,2571,Upto 5k,12290,4,Y,No,17,3,3,80,1,17,3,3,5,2,0,3.0
803,RM1466,36,36-45,No,Travel_Frequently,884,Research & Development,23,2,Medical,1,2061,3,Male,41,4,2,Laboratory Technician,4,Married,2571,Upto 5k,12290,4,Y,No,17,3,3,80,1,17,3,3,5,2,0,2.0


In [160]:
duplicates_removed_df.loc[duplicates_removed_df['EmpID'] == 'RM1467']


,EmpID,Age,AgeGroup,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,SalarySlab,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
953,RM1467,39,36-45,No,Travel_Rarely,613,Research & Development,6,1,Medical,1,2062,4,Male,42,2,3,Healthcare Representative,1,Married,9991,5k-10k,21457,4,Y,No,15,3,1,80,1,9,5,3,7,7,1,7.0
955,RM1467,39,36-45,No,Travel_Rarely,613,Research & Development,6,1,Medical,1,2062,4,Male,42,2,3,Healthcare Representative,1,Married,9991,5k-10k,21457,4,Y,No,15,3,1,80,1,9,5,3,7,7,1,1.0


In [161]:
print(f'Shape of dataframe before removing duplicate EmpID rows: {duplicates_removed_df.shape}')

duplicates_EmpID_removed_df = duplicates_removed_df.loc[~duplicates_removed_df.index.isin([161, 803, 955])]

print(f'Shape of dataframe after removing duplicate EmpID rows: {duplicates_EmpID_removed_df.shape}')


Shape of dataframe before removing duplicate EmpID rows: (1473, 38)
Shape of dataframe after removing duplicate EmpID rows: (1470, 38)


# Preprocessing: Convert Attrition to '1's and '0's

In [162]:
attrition1And0_df = duplicates_EmpID_removed_df.copy()

attrition1And0_df['Attrition'].unique()

array(['Yes', 'No'], dtype=object)

In [163]:
print(attrition1And0_df.shape)
print(attrition1And0_df.info())
attrition1And0_df.head()

(1470, 38)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1470 entries, 0 to 1479
Data columns (total 38 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   EmpID                     1470 non-null   object 
 1   Age                       1470 non-null   int64  
 2   AgeGroup                  1470 non-null   object 
 3   Attrition                 1470 non-null   object 
 4   BusinessTravel            1470 non-null   object 
 5   DailyRate                 1470 non-null   int64  
 6   Department                1470 non-null   object 
 7   DistanceFromHome          1470 non-null   int64  
 8   Education                 1470 non-null   int64  
 9   EducationField            1470 non-null   object 
 10  EmployeeCount             1470 non-null   int64  
 11  EmployeeNumber            1470 non-null   int64  
 12  EnvironmentSatisfaction   1470 non-null   int64  
 13  Gender                    1470 non-null   object 
 1

,EmpID,Age,AgeGroup,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,SalarySlab,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,RM297,18,18-25,Yes,Travel_Rarely,230,Research & Development,3,3,Life Sciences,1,405,3,Male,54,3,1,Laboratory Technician,3,Single,1420,Upto 5k,25233,1,Y,No,13,3,3,80,0,0,2,3,0,0,0,0.0
1,RM302,18,18-25,No,Travel_Rarely,812,Sales,10,3,Medical,1,411,4,Female,69,2,1,Sales Representative,3,Single,1200,Upto 5k,9724,1,Y,No,12,3,1,80,0,0,2,3,0,0,0,0.0
2,RM458,18,18-25,Yes,Travel_Frequently,1306,Sales,5,3,Marketing,1,614,2,Male,69,3,1,Sales Representative,2,Single,1878,Upto 5k,8059,1,Y,Yes,14,3,4,80,0,0,3,3,0,0,0,0.0
3,RM728,18,18-25,No,Non-Travel,287,Research & Development,5,2,Life Sciences,1,1012,2,Male,73,3,1,Research Scientist,4,Single,1051,Upto 5k,13493,1,Y,No,15,3,4,80,0,0,2,3,0,0,0,0.0
4,RM829,18,18-25,Yes,Non-Travel,247,Research & Development,8,1,Medical,1,1156,3,Male,80,3,1,Laboratory Technician,3,Single,1904,Upto 5k,13556,1,Y,No,12,3,4,80,0,0,0,3,0,0,0,0.0


In [164]:
attrition1And0_df['Attrition'] = attrition1And0_df['Attrition'].replace('No',"0")
attrition1And0_df['Attrition'] = attrition1And0_df['Attrition'].replace('Yes',"1")
attrition1And0_df['Attrition'] = attrition1And0_df['Attrition'].astype('int64')
print(attrition1And0_df.shape)
print(attrition1And0_df.info())
attrition1And0_df.head()

(1470, 38)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1470 entries, 0 to 1479
Data columns (total 38 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   EmpID                     1470 non-null   object 
 1   Age                       1470 non-null   int64  
 2   AgeGroup                  1470 non-null   object 
 3   Attrition                 1470 non-null   int64  
 4   BusinessTravel            1470 non-null   object 
 5   DailyRate                 1470 non-null   int64  
 6   Department                1470 non-null   object 
 7   DistanceFromHome          1470 non-null   int64  
 8   Education                 1470 non-null   int64  
 9   EducationField            1470 non-null   object 
 10  EmployeeCount             1470 non-null   int64  
 11  EmployeeNumber            1470 non-null   int64  
 12  EnvironmentSatisfaction   1470 non-null   int64  
 13  Gender                    1470 non-null   object 
 1

,EmpID,Age,AgeGroup,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,SalarySlab,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,RM297,18,18-25,1,Travel_Rarely,230,Research & Development,3,3,Life Sciences,1,405,3,Male,54,3,1,Laboratory Technician,3,Single,1420,Upto 5k,25233,1,Y,No,13,3,3,80,0,0,2,3,0,0,0,0.0
1,RM302,18,18-25,0,Travel_Rarely,812,Sales,10,3,Medical,1,411,4,Female,69,2,1,Sales Representative,3,Single,1200,Upto 5k,9724,1,Y,No,12,3,1,80,0,0,2,3,0,0,0,0.0
2,RM458,18,18-25,1,Travel_Frequently,1306,Sales,5,3,Marketing,1,614,2,Male,69,3,1,Sales Representative,2,Single,1878,Upto 5k,8059,1,Y,Yes,14,3,4,80,0,0,3,3,0,0,0,0.0
3,RM728,18,18-25,0,Non-Travel,287,Research & Development,5,2,Life Sciences,1,1012,2,Male,73,3,1,Research Scientist,4,Single,1051,Upto 5k,13493,1,Y,No,15,3,4,80,0,0,2,3,0,0,0,0.0
4,RM829,18,18-25,1,Non-Travel,247,Research & Development,8,1,Medical,1,1156,3,Male,80,3,1,Laboratory Technician,3,Single,1904,Upto 5k,13556,1,Y,No,12,3,4,80,0,0,0,3,0,0,0,0.0


In [165]:
attrition1And0_df.corr()

C:\Users\T852\AppData\Local\Temp\ipykernel_45704\1342716093.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  attrition1And0_df.corr()


,Age,Attrition,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,MonthlyRate,NumCompaniesWorked,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
Age,1.000000,-0.159205,0.010661,-0.001686,0.208034,NaN,-0.010145,0.010146,0.024287,0.029820,0.509604,-0.004892,0.497855,0.028051,0.299635,0.003634,0.001904,0.053535,NaN,0.037510,0.680381,-0.019621,-0.021490,0.311309,0.212901,0.216513,0.210199
Attrition,-0.159205,1.000000,-0.056652,0.077924,-0.031373,NaN,-0.010577,-0.103369,-0.006846,-0.130016,-0.169105,-0.103481,-0.159840,0.015170,0.043494,-0.013478,0.002889,-0.045872,NaN,-0.137145,-0.171063,-0.059478,-0.063939,-0.134392,-0.160545,-0.033019,-0.162628
DailyRate,0.010661,-0.056652,1.000000,-0.004985,-0.016806,NaN,-0.050990,0.018355,0.023381,0.046135,0.002966,0.030571,0.007707,-0.032182,0.038153,0.022704,0.000473,0.007846,NaN,0.042143,0.014515,0.002453,-0.037848,-0.034055,0.009932,-0.033229,-0.022880
DistanceFromHome,-0.001686,0.077924,-0.004985,1.000000,0.021042,NaN,0.032916,-0.016075,0.031131,0.008783,0.005303,-0.003669,-0.017014,0.027473,-0.029251,0.040235,0.027110,0.006557,NaN,0.044872,0.004628,-0.036942,-0.026556,0.009508,0.018845,0.010029,0.013031
Education,0.208034,-0.031373,-0.016806,0.021042,1.000000,NaN,0.042070,-0.027128,0.016775,0.042438,0.101589,-0.011296,0.094961,-0.026084,0.126317,-0.011111,-0.024539,-0.009118,NaN,0.018422,0.148280,-0.025100,0.009819,0.069114,0.060236,0.054254,0.068360
EmployeeCount,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EmployeeNumber,-0.010145,-0.010577,-0.050990,0.032916,0.042070,NaN,1.000000,0.017621,0.035179,-0.006888,-0.018519,-0.046247,-0.014829,0.012648,-0.001251,-0.012944,-0.020359,-0.069861,NaN,0.062227,-0.014365,0.023603,0.010309,-0.011240,-0.008416,-0.009019,-0.007140
EnvironmentSatisfaction,0.010146,-0.103369,0.018355,-0.016075,-0.027128,NaN,0.017621,1.000000,-0.049857,-0.008278,0.001212,-0.006784,-0.006259,0.037600,0.012594,-0.031701,-0.029548,0.007665,NaN,0.003432,-0.002693,-0.019359,0.027627,0.001458,0.018007,0.016194,0.005164
HourlyRate,0.024287,-0.006846,0.023381,0.031131,0.016775,NaN,0.035179,-0.049857,1.000000,0.042861,-0.027853,-0.071335,-0.015794,-0.015297,0.022157,-0.009062,-0.002172,0.001330,NaN,0.050263,-0.002334,-0.008548,-0.004607,-0.019582,-0.024106,-0.026716,-0.006402
JobInvolvement,0.029820,-0.130016,0.046135,0.008783,0.042438,NaN,-0.006888,-0.008278,0.042861,1.000000,-0.012630,-0.021476,-0.015271,-0.016322,0.015012,-0.017205,-0.029071,0.034297,NaN,0.021523,-0.005533,-0.015338,-0.014617,-0.021355,0.008717,-0.024184,0.038817


In [166]:
attrition1And0_df.corr().to_csv('./DataFiles/hr_analysis_correlation_after_scaling_Attrition.csv', index=False)

C:\Users\T852\AppData\Local\Temp\ipykernel_45704\1964535484.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  attrition1And0_df.corr().to_csv('./DataFiles/hr_analysis_correlation_after_scaling_Attrition.csv', index=False)


In [167]:
filepath = './DataFiles/hr_analysis_after_scaling_Attrition.csv'
attrition1And0_df.to_csv(filepath, index = False)

In [168]:
filepath = './DataFiles/hr_analysis_after_scaling_Attrition.csv'
attrition1And0_df = pd.read_csv(filepath)
attrition1And0_df

,EmpID,Age,AgeGroup,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,SalarySlab,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,RM297,18,18-25,1,Travel_Rarely,230,Research & Development,3,3,Life Sciences,1,405,3,Male,54,3,1,Laboratory Technician,3,Single,1420,Upto 5k,25233,1,Y,No,13,3,3,80,0,0,2,3,0,0,0,0.0
1,RM302,18,18-25,0,Travel_Rarely,812,Sales,10,3,Medical,1,411,4,Female,69,2,1,Sales Representative,3,Single,1200,Upto 5k,9724,1,Y,No,12,3,1,80,0,0,2,3,0,0,0,0.0
2,RM458,18,18-25,1,Travel_Frequently,1306,Sales,5,3,Marketing,1,614,2,Male,69,3,1,Sales Representative,2,Single,1878,Upto 5k,8059,1,Y,Yes,14,3,4,80,0,0,3,3,0,0,0,0.0
3,RM728,18,18-25,0,Non-Travel,287,Research & Development,5,2,Life Sciences,1,1012,2,Male,73,3,1,Research Scientist,4,Single,1051,Upto 5k,13493,1,Y,No,15,3,4,80,0,0,2,3,0,0,0,0.0
4,RM829,18,18-25,1,Non-Travel,247,Research & Development,8,1,Medical,1,1156,3,Male,80,3,1,Laboratory Technician,3,Single,1904,Upto 5k,13556,1,Y,No,12,3,4,80,0,0,0,3,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,RM412,60,55+,0,Travel_Rarely,422,Research & Development,7,3,Life Sciences,1,549,1,Female,41,3,5,Manager,1,Married,19566,15k+,3854,5,Y,No,11,3,4,80,0,33,5,1,29,8,11,10.0
1466,RM428,60,55+,0,Travel_Frequently,1499,Sales,28,3,Marketing,1,573,3,Female,80,2,3,Sales Executive,1,Married,10266,10k-15k,2845,4,Y,No,19,3,4,80,0,22,5,4,18,13,13,11.0
1467,RM537,60,55+,0,Travel_Rarely,1179,Sales,16,4,Marketing,1,732,1,Male,84,3,2,Sales Executive,1,Single,5405,5k-10k,11924,8,Y,No,14,3,4,80,0,10,1,3,2,2,2,2.0
1468,RM880,60,55+,0,Travel_Rarely,696,Sales,7,4,Marketing,1,1233,2,Male,52,4,2,Sales Executive,4,Divorced,5220,5k-10k,10893,0,Y,Yes,18,3,2,80,1,12,3,3,11,7,1,9.0


# Preprocessing: Remove Columns With Only Single Value

In [169]:
removed_singlets_df = attrition1And0_df.copy()
# Create a dictionary with column names and their unique value counts
unique_counts = {col:application_df[col].nunique() for col in removed_singlets_df.columns}

# Convert the dictionary to a DataFrame
unique_counts_df = pd.DataFrame(list(unique_counts.items()), columns=['Column Name', 'Unique Values'])

# Display the new DataFrame
print(unique_counts_df)


                 Column Name  Unique Values
0                      EmpID           1470
1                        Age             43
2                   AgeGroup              5
3                  Attrition              2
4             BusinessTravel              4
5                  DailyRate            886
6                 Department              3
7           DistanceFromHome             29
8                  Education              5
9             EducationField              6
10             EmployeeCount              1
11            EmployeeNumber           1470
12   EnvironmentSatisfaction              4
13                    Gender              2
14                HourlyRate             71
15            JobInvolvement              4
16                  JobLevel              5
17                   JobRole              9
18           JobSatisfaction              4
19             MaritalStatus              3
20             MonthlyIncome           1349
21                SalarySlab    

In [170]:
print(removed_singlets_df.shape)

(1470, 38)


In [171]:
# Drop columns with only one value

# Identify columns with only one unique value
cols_to_drop = [col for col in removed_singlets_df.columns if removed_singlets_df[col].nunique() == 1]

# Drop these columns from the DataFrame
removed_singlets_df.drop(cols_to_drop, axis=1, inplace=True)
print(removed_singlets_df.columns)
removed_singlets_df.shape


Index(['EmpID', 'Age', 'AgeGroup', 'Attrition', 'BusinessTravel', 'DailyRate',
       'Department', 'DistanceFromHome', 'Education', 'EducationField',
       'EmployeeNumber', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'MonthlyIncome', 'SalarySlab', 'MonthlyRate',
       'NumCompaniesWorked', 'OverTime', 'PercentSalaryHike',
       'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager'],
      dtype='object')


(1470, 35)

# Create Neural Network Model 1

In [108]:
# Remove outliers using 1.5 * IQR Rule then Create Neural Network Model from data
start_threshold = 1000
end_threshold = 1001
max_epoch = 10
model_name = 'Model 1'
important_info = 'Using 1.5*IQR'


columns = [['DailyRate', 'DistanceFromHome']]
for column in columns:
    df = removed_singlets_df.copy()

    # break
    run_hyperparameters(optimization_title=f'{model_name}: {important_info}: try WITHOUT dropping EmpID and EmployeeNumber and use 1.5 * IQR outlier restrictions; \
"{column}" are the only columns with outliers removed; try hyperparameters \
across thresholds {start_threshold} to {end_threshold} excluding {end_threshold} with max_epoch {max_epoch}',
                        df_with_preprocessing=df, 
                        df_without_some_preprocessing_for_test_data=df, 
                        columns_to_remove_outliers_from=column, 
                        begin_threshold=start_threshold, 
                        end_threshold=end_threshold, 
                        column_to_remove='', 
                        max_epochs=max_epoch, 
                        restrict_columns_to_interquartile=True, 
                        use_equal_number_outliers_as_signal_to_bypass_z_score_threshold_analysis=False, 
                        run_model_against_X_test_scaled_with_outliers=True)

# Make a beep sound when code is finished running
winsound.Beep(freq, duration)

Trial 60 Complete [00h 00m 05s]
val_accuracy: 1.0

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 05m 05s
1/1 - 0s - loss: 0.4659 - accuracy: 1.0000 - 145ms/epoch - 145ms/step
12/12 - 0s - loss: nan - accuracy: 0.4457 - 36ms/epoch - 3ms/step
[{'model_accuracy_WITHOUT_outliers_in_test_data': 1.0, 'timestamp': '20231128-214333', 'z_threshold': 1000, 'columns to remove outliers from': ['DailyRate', 'DistanceFromHome'], 'model_accuracy_WITH_outliers_in_test_data': 0.44565218687057495, 'number of columns': 1533, 'param': {'num_layers': 1, 'activation_layer_input': 'leaky_relu', 'neurons_layer_input': 1897, 'learning_rate': 0.0002965488058019691, 'tuner/epochs': 2, 'tuner/initial_epoch': 0, 'tuner/bracket': 2, 'tuner/round': 0}, 'columns': Index(['Age', 'Attrition', 'DailyRate', 'DistanceFromHome', 'Education',
       'EmployeeNumber', 'EnvironmentSatisfaction', 'HourlyRate',
       'JobInvolvement', 'JobLevel',
       ...
       'JobRole_Sales Representative', 'MaritalStatus_Divorced

### Result: Neural Network Model 1
It did give a high accuracy of 100% BUT that's only because the 1.5 * IQR outlier restriction ruled out all but 3 out of 1470 rows. The accuracy of this model when outliers are included in the testing data but not in the training data is 58.42% which is 17% below 75% goal performance. The 1.5 * IQR Rule outlier restrictions make the model too fragile an unrobust to varied datasets.

# Create Neural Network Model 2

In [109]:
# Don't remove any outliers (the z-score-threshold was set to 1000 as the cut off limit to assure that no rows were remove) then Create Neural Network Model from data
start_threshold = 1000
end_threshold = 1001
max_epoch = 10
model_name = 'Model 2'
important_info = 'Using no outliers'

columns = [['DailyRate', 'DistanceFromHome']]
for column in columns:
    df = removed_singlets_df.copy()
    # break
    run_hyperparameters(optimization_title=f'{model_name}: {important_info}: try WITHOUT dropping EmpID and EmployeeNumber and using z-scores (not IQR) that are so high \
that NO outliers are found (so keeping all rows); "{column}" are the \
only columns with outliers removed; try hyperparameters across thresholds {start_threshold} to \
{end_threshold} excluding {end_threshold} with max_epoch {max_epoch}',
                        df_with_preprocessing=df,
                        df_without_some_preprocessing_for_test_data=df,
                        columns_to_remove_outliers_from=column, 
                        begin_threshold=start_threshold,
                        end_threshold=end_threshold,
                        column_to_remove='',
                        max_epochs=max_epoch,
                        restrict_columns_to_interquartile=False,
                        use_equal_number_outliers_as_signal_to_bypass_z_score_threshold_analysis=False,
                        run_model_against_X_test_scaled_with_outliers=True)

# Make a beep sound when code is finished running
winsound.Beep(freq, duration)

Trial 60 Complete [00h 00m 11s]
val_accuracy: 0.8396739363670349

Best val_accuracy So Far: 0.8396739363670349
Total elapsed time: 00h 15m 26s
12/12 - 0s - loss: nan - accuracy: 0.8397 - 179ms/epoch - 15ms/step
12/12 - 0s - loss: nan - accuracy: 0.8397 - 30ms/epoch - 3ms/step
[{'model_accuracy_WITHOUT_outliers_in_test_data': 0.8396739363670349, 'timestamp': '20231128-220042', 'z_threshold': 1000, 'columns to remove outliers from': ['DailyRate', 'DistanceFromHome'], 'model_accuracy_WITH_outliers_in_test_data': 0.8396739363670349, 'number of columns': 1533, 'param': {'num_layers': 1, 'activation_layer_input': 'leaky_relu', 'neurons_layer_input': 1897, 'learning_rate': 0.0002965488058019691, 'tuner/epochs': 2, 'tuner/initial_epoch': 0, 'tuner/bracket': 2, 'tuner/round': 0}, 'columns': Index(['Age', 'Attrition', 'DailyRate', 'DistanceFromHome', 'Education',
       'EmployeeNumber', 'EnvironmentSatisfaction', 'HourlyRate',
       'JobInvolvement', 'JobLevel',
       ...
       'JobRole_Sale

### Result: Neural Network Model 2
Model 2 had an accuracy of 83.967% with 1470 rows out 1470 rows (no rows were removed). But the number columns/features were 1533 b/c of the EmpID and EmployeeNumber, so, let's see if removing those columns will reduce unnecessary complexity and maintain or increase accuracy and reduce processing time.

# Create Neural Network Model 2.Z-Series

In [46]:
# Create Neural Network Models for training data that removes outliers from as defined by z-score starting at 1 and ending at 11 (not including 11).
# Note that that if a z-score doesn't remove any new rows as outliers comparied to the number of outliers in the previously run model
# then that z-score model is skipped b/c the training data will be identical to the previous model's training and so reproduces the 
# exact same results. (This is why "use_equal_number_outliers_as_signal_to_bypass_z_score_threshold_analysis" is set to True).
start_threshold = 1
end_threshold = 11
max_epoch = 10

columns = [
            'Age', 
           'DailyRate', 
           'DistanceFromHome', 
           'Education', 
           'EnvironmentSatisfaction', 
           'HourlyRate', 
           'JobInvolvement', 
           'JobLevel', 
           'JobSatisfaction', 
           'MonthlyIncome', 
           'NumCompaniesWorked',
           'PercentSalaryHike',
           'PerformanceRating',
           'RelationshipSatisfaction',
           'StockOptionLevel',
           'TotalWorkingYears',
           'TrainingTimesLastYear',
           'WorkLifeBalance',
           'YearsAtCompany',
           'YearsInCurrentRole',
           'YearsSinceLastPromotion',
           'YearsWithCurrManager'
           ]
for column in columns:
    df = removed_singlets_df.copy()
    columns_to_remove_outliers_from = []
    columns_to_remove_outliers_from.append(column)
    print(column)
    # break
    run_hyperparameters(optimization_title=f'try WITHOUT dropping EmpID and EmployeeNumber and using z-scores (not IQR) that are so high \
that NO outliers are found (so keeping all rows); "{columns_to_remove_outliers_from}" are the \
only columns with outliers removed; try hyperparameters across thresholds {start_threshold} to \
{end_threshold} excluding {end_threshold} with max_epoch {max_epoch}',
                        df_with_preprocessing=df,
                        df_without_some_preprocessing_for_test_data=df,
                        columns_to_remove_outliers_from=[column], 
                        begin_threshold=start_threshold,
                        end_threshold=end_threshold,
                        column_to_remove='',
                        max_epochs=max_epoch,
                        restrict_columns_to_interquartile=False,
                        use_equal_number_outliers_as_signal_to_bypass_z_score_threshold_analysis=True,
                        run_model_against_X_test_scaled_with_outliers=True)

# Make a beep sound when code is finished running
winsound.Beep(freq, duration)

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.8416666388511658

Best val_accuracy So Far: 0.8416666388511658
Total elapsed time: 00h 00m 16s

Search: Running Trial #6

Value             |Best Value So Far |Hyperparameter
3                 |1                 |num_layers
tanh              |leaky_relu        |activation_layer_input
1587              |1897              |neurons_layer_input
0.00065625        |0.00029655        |learning_rate
relu              |None              |activation_layer_1
2722              |None              |neurons_layer_1
relu              |None              |activation_layer_2
1681              |None              |neurons_layer_2
relu              |None              |activation_layer_3
1525              |None              |neurons_layer_3
relu              |None              |activation_layer_4
2998              |None              |neurons_layer_4
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2    

KeyboardInterrupt: 

NOTE: The "Error Message" above was deliberately invoked ("KeyboardInterrupt") simply because the processing time was taking too long

### Result: Neural Network Model2.Z-Series
Neural Network Model 2.Z-Series had the highest accuracy 85.946% for EnvironmentSatisfaction with a z-score threshold of 1. Not all of these model2.Z-Series were created b/c each model had an input of 1533 features making the processing time too long, so, the effort was abandoned after making 35 models at varying z-score thresholds for columns in the following list: <br>
'Age', <br>
'DailyRate', <br>
'DistanceFromHome', <br>
'Education', <br>
'EnvironmentSatisfaction', <br>
'HourlyRate', <br>
'JobInvolvement', <br>
'JobLevel', <br>
'JobSatisfaction', <br>
'MonthlyIncome', <br>
'NumCompaniesWorked'

Note: while the searches could in theory go as high as a z-score of 11, none of the columns had data that varied that significantly so the process (and number of models) was shorten by a dramatic amount.

# Preprocessing: Remove ID Columns

In [172]:
# Remove id column
print(removed_singlets_df.shape)
remove_id_df = removed_singlets_df.copy()
remove_id_df.drop(['EmpID'], axis=1, inplace=True)
remove_id_df.shape

(1470, 35)


(1470, 34)

In [173]:
# Remove id column
remove_id_df.drop(['EmployeeNumber'], axis=1, inplace=True)
remove_id_df.shape

(1470, 33)

# Create Neural Network Model 3

In [121]:
# Create Neural Network Model with dropped columns EmpID and EmployeeNumber use 1.5 * IQR Rule
start_threshold = 1000
end_threshold = 1001
max_epoch = 10
model_name = 'Model 3'
important_info = 'Dropped EmpID and EmployeeNumber, Use 1.5 * IQR Rule'

columns = [['DailyRate', 'DistanceFromHome']]
for column in columns:
    df = remove_id_df.copy()
    columns_to_remove_outliers_from = []
    columns_to_remove_outliers_from.append(column)
    print(column)
    # break
    run_hyperparameters(optimization_title=f'{model_name}: {important_info}: try WITH dropping EmpID and EmployeeNumber and use 1.5 * IQR outlier restrictions; \
"{column}" are the only columns with outliers removed; try \
hyperparameters across thresholds {start_threshold} to {end_threshold} excluding \
{end_threshold} with max_epoch {max_epoch}',
                        df_with_preprocessing=df,
                        df_without_some_preprocessing_for_test_data=df,
                        columns_to_remove_outliers_from=column,
                        begin_threshold=start_threshold,
                        end_threshold=end_threshold,
                        column_to_remove='',
                        max_epochs=max_epoch,
                        restrict_columns_to_interquartile=True,
                        use_equal_number_outliers_as_signal_to_bypass_z_score_threshold_analysis=False,
                        run_model_against_X_test_scaled_with_outliers=True)

# Make a beep sound when code is finished running
winsound.Beep(freq, duration)

Trial 60 Complete [00h 00m 06s]
val_accuracy: 1.0

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 06m 04s
1/1 - 0s - loss: 0.6322 - accuracy: 1.0000 - 151ms/epoch - 151ms/step
12/12 - 0s - loss: 0.6733 - accuracy: 0.8397 - 37ms/epoch - 3ms/step
[{'model_accuracy_WITHOUT_outliers_in_test_data': 1.0, 'timestamp': '20231128-235745', 'z_threshold': 1000, 'columns to remove outliers from': ['DailyRate', 'DistanceFromHome'], 'model_accuracy_WITH_outliers_in_test_data': 0.8396739363670349, 'number of columns': 62, 'param': {'num_layers': 4, 'activation_layer_input': 'leaky_relu', 'neurons_layer_input': 81, 'learning_rate': 0.00028406107009737317, 'tuner/epochs': 2, 'tuner/initial_epoch': 0, 'tuner/bracket': 2, 'tuner/round': 0, 'activation_layer_1': 'relu', 'neurons_layer_1': 2, 'activation_layer_2': 'relu', 'neurons_layer_2': 2, 'activation_layer_3': 'relu', 'neurons_layer_3': 2}, 'columns': Index(['Age', 'Attrition', 'DailyRate', 'DistanceFromHome', 'Education',
       'EnvironmentSa

### Result: Neural Network Model3
Neural Network Model3 did give a high accuracy of 100% BUT that's only b/c the 1.5 * IQR outlier restriction ruled out all but 75 out of 1470 rows. The accuracy of this model when outliers are included in the testing data (but not in the training data) is 83.967% but again b/c the number of rows it that it was trained on were only 75 out of 1470, other models are likely to be equally as accurate and far more robust in handling greater variety of data. Having so few rows also may increase the chances of overfitting.

# Create Neural Network Model 4

In [188]:
# Create Neural Network Model with dropped columns EmpID and EmployeeNumber and do NOT use 1.5 * IQR Rule
start_threshold = 1000
end_threshold = 1001
max_epoch = 10
model_name = 'Model 4'
important_info = 'Dropped EmpID and EmployeeNumber: no outliers'

# given that z-scores are so high '1000' and '1001'--> 'columns' is just a dummy/placeholder variable
columns = ['DailyRate'] 
for column in columns:
    df = remove_id_df.copy()
    # break
    run_hyperparameters(optimization_title=f'{model_name}: {important_info}: try WITH dropping EmpID and EmployeeNumber and using z-scores (not IQR) that are so high \
that NO outliers are found (so keeping all rows); \
"{column}" are the only columns with outliers removed; try \
hyperparameters across thresholds {start_threshold} to {end_threshold} excluding \
{end_threshold} with max_epoch {max_epoch}',
                        df_with_preprocessing=df,
                        df_without_some_preprocessing_for_test_data=df,
                        columns_to_remove_outliers_from=[column],
                        begin_threshold=start_threshold,
                        end_threshold=end_threshold,
                        column_to_remove='',
                        max_epochs=max_epoch,
                        restrict_columns_to_interquartile=False,
                        use_equal_number_outliers_as_signal_to_bypass_z_score_threshold_analysis=False,
                        run_model_against_X_test_scaled_with_outliers=True)

# Make a beep sound when code is finished running
winsound.Beep(freq, duration)

Trial 60 Complete [00h 00m 08s]
val_accuracy: 0.8396739363670349

Best val_accuracy So Far: 0.8396739363670349
Total elapsed time: 00h 08m 10s
12/12 - 0s - loss: nan - accuracy: 0.8397 - 119ms/epoch - 10ms/step
12/12 - 0s - loss: nan - accuracy: 0.8397 - 23ms/epoch - 2ms/step
[{'model_accuracy_WITHOUT_outliers_in_test_data': 0.8396739363670349, 'timestamp': '20231129-043115', 'z_threshold': 1000, 'columns to remove outliers from': ['DailyRate'], 'model_accuracy_WITH_outliers_in_test_data': 0.8396739363670349, 'number of columns': 62, 'param': {'num_layers': 1, 'activation_layer_input': 'leaky_relu', 'neurons_layer_input': 76, 'learning_rate': 0.0002965488058019691, 'tuner/epochs': 2, 'tuner/initial_epoch': 0, 'tuner/bracket': 2, 'tuner/round': 0}, 'columns': Index(['Age', 'Attrition', 'DailyRate', 'DistanceFromHome', 'Education',
       'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement', 'JobLevel',
       'JobSatisfaction', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',


### Result: Neural Network Model 4
Neural Network Model4 had an accuracy of 83.967% with 1470 rows out 1470 rows (no rows were removed). Now the number columns/features were 62 b/c the EmpID and EmployeeNumber were removed. Now, that the number of columns are far fewer let's vary the z-score thresholds with a new z-series.

# Create Neural Network Model 4.Z-Series

In [48]:
# Create Neural Network Models for training data that removes outliers from as defined by z-score starting at 1 and ending at 11 (not including 11).
# Note that that if a z-score doesn't remove any new rows as outliers comparied to the number of outliers in the previously run model
# then that z-score model is skipped b/c the training data will be identical to the previous model's training and so reproduces the 
# exact same results. (This is why "use_equal_number_outliers_as_signal_to_bypass_z_score_threshold_analysis" is set to True).
start_threshold = 1
end_threshold = 11
max_epoch = 10
model_name = 'Model 4.Z-Series'

columns = [
           'Age', 
           'DailyRate', 
           'DistanceFromHome', 
           'Education', 
           'EnvironmentSatisfaction', 
           'HourlyRate', 
           'JobInvolvement', 
           'JobLevel', 
           'JobSatisfaction', 
           'MonthlyIncome', 
           'NumCompaniesWorked',
           'PercentSalaryHike',
           'PerformanceRating',
           'RelationshipSatisfaction',
           'StockOptionLevel',
           'TotalWorkingYears',
           'TrainingTimesLastYear',
           'WorkLifeBalance',
           'YearsAtCompany',
           'YearsInCurrentRole',
           'YearsSinceLastPromotion',
           'YearsWithCurrManager'
           ]
for column in columns:
    df = remove_id_df.copy()
    important_model_info = column
    run_hyperparameters(optimization_title=f'{model_name}: {important_model_info}: try WITH dropping EmpID and EmployeeNumber and z-scores will keep reading for\
z-score details; "{column}" are the \
only columns with outliers removed; try hyperparameters across z-score thresholds {start_threshold} to \
{end_threshold} excluding {end_threshold} with max_epoch {max_epoch}',
                        df_with_preprocessing=df,
                        df_without_some_preprocessing_for_test_data=df,
                        columns_to_remove_outliers_from=[column], 
                        begin_threshold=start_threshold,
                        end_threshold=end_threshold,
                        column_to_remove='',
                        max_epochs=max_epoch,
                        restrict_columns_to_interquartile=False,
                        use_equal_number_outliers_as_signal_to_bypass_z_score_threshold_analysis=True,
                        run_model_against_X_test_scaled_with_outliers=True)

# Make a beep sound when code is finished running
winsound.Beep(freq, duration)

Trial 60 Complete [00h 00m 17s]
val_accuracy: 0.8396739363670349

Best val_accuracy So Far: 0.8396739363670349
Total elapsed time: 00h 16m 27s
12/12 - 0s - loss: nan - accuracy: 0.8397 - 124ms/epoch - 10ms/step
12/12 - 0s - loss: nan - accuracy: 0.8397 - 24ms/epoch - 2ms/step
['YearsSinceLastPromotion']
<class 'list'>
Outlier rows:
Int64Index([], dtype='int64')
0
['YearsSinceLastPromotion']
<class 'list'>
Outlier rows:
Int64Index([], dtype='int64')
0
['YearsSinceLastPromotion']
<class 'list'>
Outlier rows:
Int64Index([], dtype='int64')
0
['YearsSinceLastPromotion']
<class 'list'>
Outlier rows:
Int64Index([], dtype='int64')
0
['YearsSinceLastPromotion']
<class 'list'>
Outlier rows:
Int64Index([], dtype='int64')
0
['YearsSinceLastPromotion']
<class 'list'>
Outlier rows:
Int64Index([], dtype='int64')
0
[{'model_accuracy_WITHOUT_outliers_in_test_data': 0.8396739363670349, 'timestamp': '20231127-200104', 'z_threshold': 5, 'columns to remove outliers from': ['YearsAtCompany'], 'model_accurac

### Result: Neural Network Model 4.Z-Series
Neural Network Model4.Z-Series had the highest accuracy 85.946% for EnvironmentSatisfaction with a z-score threshold of 1. And unlike in Model2.Z-Series, the Model4.Z-Series was completed all models searched totalling 66 models. Like before, while the searches could in theory go as high as a z-score of 11, none of the columns had data that varied that significantly so the process (and number of models) was shorten by a dramatic amount.

# Neural Network Model 4.Z-Series-RESULT1-84%-Z-Score-1

In [111]:
start_threshold = 1
end_threshold = 2
max_epoch = 10
model_name = 'Model 4.Z-Series-RESULT2-84%-Z-Score-1'
important_model_info = 'Use z-score-threshold 1 Only on increased-accuracy-columns of 84+%'

columns = [['Age', 
            'DailyRate', 
            'DistanceFromHome', 
            'EnvironmentSatisfaction', 
            'JobInvolvement',
            'NumCompaniesWorked', 
            'PercentSalaryHike',
            'RelationshipSatisfaction',
            'TotalWorkingYears',
            'TrainingTimesLastYear',
            'WorkLifeBalance',
            'YearsInCurrentRole']]
for column in columns:
    df = remove_id_df.copy()
    run_hyperparameters(f'{model_name}: {important_model_info}: try removing outliers as defined by unbounded data in the form of "SalarySlab" with a value equal to "15k+"; \
try WITH dropping EmpID and EmployeeNumber and using 1.5 * IQR Rule to remove outliers; \
"{column}" are the only columns with outliers removed; \
try hyperparameters across thresholds {start_threshold} to {end_threshold} excluding {end_threshold} with \
max_epoch {max_epoch}', 
                        df_with_preprocessing=df,
                        df_without_some_preprocessing_for_test_data=df,
                        columns_to_remove_outliers_from=column, 
                        begin_threshold=start_threshold,
                        end_threshold=end_threshold,
                        column_to_remove='',
                        max_epochs=max_epoch,
                        restrict_columns_to_interquartile=False,
                        use_equal_number_outliers_as_signal_to_bypass_z_score_threshold_analysis=False,
                        run_model_against_X_test_scaled_with_outliers=True)

# Make a beep sound when code is finished running
winsound.Beep(freq, duration)

Trial 60 Complete [00h 00m 05s]
val_accuracy: 0.5

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 05m 12s
1/1 - 0s - loss: 0.5963 - accuracy: 1.0000 - 213ms/epoch - 213ms/step
12/12 - 0s - loss: 0.7403 - accuracy: 0.4212 - 187ms/epoch - 16ms/step
[{'model_accuracy_WITHOUT_outliers_in_test_data': 1.0, 'timestamp': '20231128-222335', 'z_threshold': 1, 'columns to remove outliers from': ['Age', 'DailyRate', 'DistanceFromHome', 'EnvironmentSatisfaction', 'JobInvolvement', 'NumCompaniesWorked', 'PercentSalaryHike', 'RelationshipSatisfaction', 'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsInCurrentRole'], 'model_accuracy_WITH_outliers_in_test_data': 0.4211956560611725, 'number of columns': 62, 'param': {'num_layers': 3, 'activation_layer_input': 'tanh', 'neurons_layer_input': 86, 'learning_rate': 0.0008894905887153519, 'activation_layer_1': 'relu', 'neurons_layer_1': 14, 'activation_layer_2': 'leaky_relu', 'neurons_layer_2': 45, 'activation_layer_3': 'leaky_re

### Result: Neural Network Model 4.Z-Series-RESULT1-84%-Z-Score-1
Neural Network Model 4.Z-Series-RESULT1-84%-Z-Score-1 was a model created by removing all z-score threshold 1 associated with EVERY column that got ABOVE 84% accuracy when individually removing outliers. The accuracy for Model 4.Z-Series-RESULT1-84%-Z-Score-1 is 100%, HOWEVER, the z-score-threshold of 1 across the <strong>12</strong> different columns left only 7 rows out of 1470 rows. So, the model is too fragile to be considered reliable. And when tested against data without outliers removed its accuracy drops to 42.120% which is quite low.

# Neural Network Model 4.Z-Series-RESULT2-85%-Z-Score-1

In [112]:
start_threshold = 1
end_threshold = 2
max_epoch = 10
model_name = 'Model 4.Z-Series-RESULT2-85%-Z-Score-1'
important_model_info = 'Use z-score-threshold 1 Only on increased-accuracy-columns of 85+%'

columns = [['Age', 
            'EnvironmentSatisfaction', 
            'JobInvolvement',
            'TotalWorkingYears',
            'WorkLifeBalance',
            'YearsInCurrentRole']]
for column in columns:
    df = remove_id_df.copy()
    run_hyperparameters(f'{model_name}: {important_model_info}: try removing outliers as defined by unbounded data in the form of "SalarySlab" with a value equal to "15k+"; \
try WITH dropping EmpID and EmployeeNumber and using 1.5 * IQR Rule to remove outliers; \
"{column}" are the only columns with outliers removed; \
try hyperparameters across thresholds {start_threshold} to {end_threshold} excluding {end_threshold} with \
max_epoch {max_epoch}', 
                        df_with_preprocessing=df,
                        df_without_some_preprocessing_for_test_data=df,
                        columns_to_remove_outliers_from=column, 
                        begin_threshold=start_threshold,
                        end_threshold=end_threshold,
                        column_to_remove='',
                        max_epochs=max_epoch,
                        restrict_columns_to_interquartile=False,
                        use_equal_number_outliers_as_signal_to_bypass_z_score_threshold_analysis=False,
                        run_model_against_X_test_scaled_with_outliers=True)

# Make a beep sound when code is finished running
winsound.Beep(freq, duration)

Trial 60 Complete [00h 00m 05s]
val_accuracy: 0.8928571343421936

Best val_accuracy So Far: 0.8928571343421936
Total elapsed time: 00h 05m 21s
1/1 - 0s - loss: nan - accuracy: 0.8929 - 116ms/epoch - 116ms/step
12/12 - 0s - loss: nan - accuracy: 0.8397 - 35ms/epoch - 3ms/step
[{'model_accuracy_WITHOUT_outliers_in_test_data': 0.8928571343421936, 'timestamp': '20231128-223548', 'z_threshold': 1, 'columns to remove outliers from': ['Age', 'EnvironmentSatisfaction', 'JobInvolvement', 'TotalWorkingYears', 'WorkLifeBalance', 'YearsInCurrentRole'], 'model_accuracy_WITH_outliers_in_test_data': 0.8396739363670349, 'number of columns': 62, 'param': {'num_layers': 1, 'activation_layer_input': 'leaky_relu', 'neurons_layer_input': 76, 'learning_rate': 0.0002965488058019691, 'tuner/epochs': 2, 'tuner/initial_epoch': 0, 'tuner/bracket': 2, 'tuner/round': 0}, 'columns': Index(['Age', 'Attrition', 'DailyRate', 'DistanceFromHome', 'Education',
       'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolveme

### Result: Neural Network Model 4.Z-Series-RESULT2-85%-Z-Score-1
Neural Network Model 4.Z-Series-RESULT2-85%-Z-Score-1 was a model created by removing all z-score threshold 1 associated with EVERY column that got ABOVE 85% accuracy when individually removing outliers. The accuracy for Model 4.Z-Series-RESULT1-84%-Z-Score-1 is 89.286%, HOWEVER, the z-score-threshold of 1 across the <strong>6</strong> different columns left only 110 rows out of 1470 rows. So, the model is too fragile to be considered reliable i.e. the relatively small amount of training and testing data make for a less robust model.

# Neural Network Model 4.Z-Series-RESULT3-85.5%-Z-Score-1

In [120]:
start_threshold = 1
end_threshold = 2
max_epoch = 10
model_name = 'Model 4.Z-Series-RESULT3-85%.5-Z-Score-1'
important_model_info = 'Use z-score-threshold 1 Only on increased-accuracy-columns of 85.5+%'

columns = [['EnvironmentSatisfaction', 
            'JobInvolvement',
            'WorkLifeBalance']]
for column in columns:
    df = remove_id_df.copy()
    run_hyperparameters(f'{model_name}: {important_model_info}: try removing outliers as defined by unbounded data in the form of "SalarySlab" with a value equal to "15k+"; \
try WITH dropping EmpID and EmployeeNumber and using 1.5 * IQR Rule to remove outliers; \
"{column}" are the only columns with outliers removed; \
try hyperparameters across thresholds {start_threshold} to {end_threshold} excluding {end_threshold} with \
max_epoch {max_epoch}', 
                        df_with_preprocessing=df,
                        df_without_some_preprocessing_for_test_data=df,
                        columns_to_remove_outliers_from=column, 
                        begin_threshold=start_threshold,
                        end_threshold=end_threshold,
                        column_to_remove='',
                        max_epochs=max_epoch,
                        restrict_columns_to_interquartile=False,
                        use_equal_number_outliers_as_signal_to_bypass_z_score_threshold_analysis=False,
                        run_model_against_X_test_scaled_with_outliers=True)

# Make a beep sound when code is finished running
winsound.Beep(freq, duration)

Trial 60 Complete [00h 00m 06s]
val_accuracy: 0.892307698726654

Best val_accuracy So Far: 0.892307698726654
Total elapsed time: 00h 05m 53s
3/3 - 0s - loss: nan - accuracy: 0.8923 - 114ms/epoch - 38ms/step
12/12 - 0s - loss: nan - accuracy: 0.8397 - 23ms/epoch - 2ms/step
[{'model_accuracy_WITHOUT_outliers_in_test_data': 0.892307698726654, 'timestamp': '20231128-232828', 'z_threshold': 1, 'columns to remove outliers from': ['EnvironmentSatisfaction', 'JobInvolvement', 'WorkLifeBalance'], 'model_accuracy_WITH_outliers_in_test_data': 0.8396739363670349, 'number of columns': 62, 'param': {'num_layers': 1, 'activation_layer_input': 'leaky_relu', 'neurons_layer_input': 76, 'learning_rate': 0.0002965488058019691, 'tuner/epochs': 2, 'tuner/initial_epoch': 0, 'tuner/bracket': 2, 'tuner/round': 0}, 'columns': Index(['Age', 'Attrition', 'DailyRate', 'DistanceFromHome', 'Education',
       'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement', 'JobLevel',
       'JobSatisfaction', 'MonthlyInc

### Result: Neural Network Model 4.Z-Series-RESULT3-85.5%-Z-Score-1
Neural Network Model 4.Z-Series-RESULT3-85.5%-Z-Score-1 was a model created by removing all z-score threshold 1 associated with EVERY column that got ABOVE 85.5% accuracy when individually removing outliers. The accuracy for Model 4.Z-Series-RESULT1-84%-Z-Score-1 is 89.231%, HOWEVER, the z-score-threshold of 1 across the <strong>3</strong> different columns left only 259 rows out of 1470 rows. So, the model is too fragile to be considered reliable i.e. the relatively small amount of training and testing data make for a less robust model.

# Preprocessing: Remove Unbound Data

In [44]:
remove_id_df['SalarySlab'].unique()

array(['Upto 5k', '5k-10k', '10k-15k', '15k+'], dtype=object)

In [45]:
remove_unbound_df = remove_id_df[remove_id_df['SalarySlab'] != '15k+']
remove_unbound_df['SalarySlab'].unique()

array(['Upto 5k', '5k-10k', '10k-15k'], dtype=object)

In [46]:
remove_unbound_df.shape

(1337, 33)


### (Model 5 was removed for being an irrelevant IQR related model (too fragile))

# Create Neural Network Model 6

In [107]:
start_threshold = 1000
end_threshold = 1001
max_epoch = 10
model_name = 'Model 5'

columns = ['DailyRate']
for column in columns:
    df = remove_unbound_df.copy()
    important_model_info = 'SalarySlab No 15k+'
    run_hyperparameters(f'{model_name}: {important_model_info}: try removing outliers as defined by unbounded data in the form of "SalarySlab" with \
a value equal to "15k+"; try WITH dropping EmpID and EmployeeNumber and using z-scores \
(not IQR) that are so high that NO outliers are found (so keeping all rows); \
"{column}" are the only columns with outliers removed; try \
hyperparameters across thresholds {start_threshold} to {end_threshold} excluding \
{end_threshold} with max_epoch {max_epoch}', 
                        df_with_preprocessing=df,
                        df_without_some_preprocessing_for_test_data=remove_id_df,
                        columns_to_remove_outliers_from=[column], 
                        begin_threshold=start_threshold,
                        end_threshold=end_threshold,
                        column_to_remove='',
                        max_epochs=max_epoch,
                        restrict_columns_to_interquartile=False,
                        use_equal_number_outliers_as_signal_to_bypass_z_score_threshold_analysis=False,
                        run_model_against_X_test_scaled_with_outliers=True)

# Make a beep sound when code is finished running
winsound.Beep(freq, duration)

Trial 60 Complete [00h 00m 04s]
val_accuracy: 0.8268656730651855

Best val_accuracy So Far: 0.8268656730651855
Total elapsed time: 00h 04m 20s
11/11 - 0s - loss: nan - accuracy: 0.8269 - 123ms/epoch - 11ms/step
12/12 - 0s - loss: nan - accuracy: 0.8397 - 26ms/epoch - 2ms/step
[{'model_accuracy_WITHOUT_outliers_in_test_data': 0.8268656730651855, 'timestamp': '20231128-092827', 'z_threshold': 1000, 'columns to remove outliers from': ['DailyRate'], 'model_accuracy_WITH_outliers_in_test_data': 0.8396739363670349, 'number of columns': 61, 'param': {'num_layers': 1, 'activation_layer_input': 'leaky_relu', 'neurons_layer_input': 75, 'learning_rate': 0.0002965488058019691, 'tuner/epochs': 2, 'tuner/initial_epoch': 0, 'tuner/bracket': 2, 'tuner/round': 0}, 'columns': Index(['Age', 'Attrition', 'DailyRate', 'DistanceFromHome', 'Education',
       'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement', 'JobLevel',
       'JobSatisfaction', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',


### Result: Neural Network Model 6
Model 6 had an accuracy of 82.687% with 1337 rows out 1470 rows where any row that had a SalarySlab value of '15k+' was removed b/c it's unbound data in that there is no upper limit on whatever the actual amount was (e.g. the amount could have been 16k or 30k, so, '15k+' is presumed to be an outlier for the sake of getting greater accuracy) and had an accuracy of 83.967% with 1470 rows out of 1470 rows. Let's take a step further to assess what changes in accuracy occur by changing SalarySlab to a scalar.

# Preprocessing: Scale SalarySlab

In [72]:
remove_unbound_df.dtypes


Age                           int64
AgeGroup                     object
Attrition                     int64
BusinessTravel               object
DailyRate                     int64
Department                   object
DistanceFromHome              int64
Education                     int64
EducationField               object
EnvironmentSatisfaction       int64
Gender                       object
HourlyRate                    int64
JobInvolvement                int64
JobLevel                      int64
JobRole                      object
JobSatisfaction               int64
MaritalStatus                object
MonthlyIncome                 int64
SalarySlab                   object
MonthlyRate                   int64
NumCompaniesWorked            int64
OverTime                     object
PercentSalaryHike             int64
PerformanceRating             int64
RelationshipSatisfaction      int64
StockOptionLevel              int64
TotalWorkingYears             int64
TrainingTimesLastYear       

In [73]:
# Set data to midpoint of each range
scale_SalarySlab_df = remove_unbound_df.copy()
scale_SalarySlab_df['SalarySlab'] = scale_SalarySlab_df['SalarySlab'].replace('Upto 5k',"2500")
scale_SalarySlab_df['SalarySlab'] = scale_SalarySlab_df['SalarySlab'].replace('5k-10k',"7500")
scale_SalarySlab_df['SalarySlab'] = scale_SalarySlab_df['SalarySlab'].replace('10k-15k',"12500")

# Make into scale
scale_SalarySlab_df['SalarySlab'] = scale_SalarySlab_df['SalarySlab'].astype('int64')
scale_SalarySlab_df.dtypes

Age                           int64
AgeGroup                     object
Attrition                     int64
BusinessTravel               object
DailyRate                     int64
Department                   object
DistanceFromHome              int64
Education                     int64
EducationField               object
EnvironmentSatisfaction       int64
Gender                       object
HourlyRate                    int64
JobInvolvement                int64
JobLevel                      int64
JobRole                      object
JobSatisfaction               int64
MaritalStatus                object
MonthlyIncome                 int64
SalarySlab                    int64
MonthlyRate                   int64
NumCompaniesWorked            int64
OverTime                     object
PercentSalaryHike             int64
PerformanceRating             int64
RelationshipSatisfaction      int64
StockOptionLevel              int64
TotalWorkingYears             int64
TrainingTimesLastYear       

In [74]:
scale_SalarySlab_df['SalarySlab'].unique()


array([ 2500,  7500, 12500], dtype=int64)

### (Neural Network Model 7 was removed for being IQR variant and too fragile and irrelevant)

# Create Neural Network Model 8

In [90]:
start_threshold = 1000
end_threshold = 1001
max_epoch = 10
model_name = 'Model 8'

columns = ['DailyRate']
for column in columns:
    df = scale_SalarySlab_df.copy()
    important_model_info = 'SalarySlab Scaling'
    run_hyperparameters(f'{model_name}: {important_model_info}: try scaling "SalarySlab" from string to int64 after removing outliers as defined \
by unbounded data in the form of "SalarySlab" with a value equal to "15k+"; try WITH \
dropping EmpID and EmployeeNumber and using z-scores (not IQR) that are so high that \
NO outliers are found (so keeping all rows); "{column}" are \
the only columns with outliers removed; try hyperparameters across thresholds \
{start_threshold} to {end_threshold} excluding {end_threshold} with max_epoch \
{max_epoch}', 
                        df_with_preprocessing=df,
                        df_without_some_preprocessing_for_test_data=remove_id_df,
                        columns_to_remove_outliers_from=[column], 
                        begin_threshold=start_threshold,
                        end_threshold=end_threshold,
                        column_to_remove='',
                        max_epochs=max_epoch,
                        restrict_columns_to_interquartile=False,
                        use_equal_number_outliers_as_signal_to_bypass_z_score_threshold_analysis=False,
                        run_model_against_X_test_scaled_with_outliers=False)

# Make a beep sound when code is finished running
winsound.Beep(freq, duration)

Trial 60 Complete [00h 00m 04s]
val_accuracy: 0.8268656730651855

Best val_accuracy So Far: 0.8268656730651855
Total elapsed time: 00h 03m 29s
11/11 - 0s - loss: nan - accuracy: 0.8269 - 113ms/epoch - 10ms/step
[{'model_accuracy_WITHOUT_outliers_in_test_data': 0.8268656730651855, 'timestamp': '20231128-074242', 'z_threshold': 1000, 'columns to remove outliers from': ['DailyRate'], 'model_accuracy_WITH_outliers_in_test_data': 'did not run with outliers included', 'number of columns': 59, 'param': {'num_layers': 1, 'activation_layer_input': 'leaky_relu', 'neurons_layer_input': 68, 'learning_rate': 0.0002965488058019691, 'tuner/epochs': 2, 'tuner/initial_epoch': 0, 'tuner/bracket': 2, 'tuner/round': 0}, 'columns': Index(['Age', 'Attrition', 'DailyRate', 'DistanceFromHome', 'Education',
       'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement', 'JobLevel',
       'JobSatisfaction', 'MonthlyIncome', 'SalarySlab', 'MonthlyRate',
       'NumCompaniesWorked', 'PercentSalaryHike', 'Perfo

### Result: Neural Network Model 8
 Neural Network Model 8 had an accuracy of 82.687% with 1337 rows out 1480 rows (outliers removed) and model 8 was not ran against data that included the outliers because including the outliers would change the number of columns/featuers/inputs to the model and so making the model incompatible with the outliers-included-test-data. Also, because the accuracy decreased with the scaling of SalarySlab and the removing of the unbound data (SalarySlab == '15k+'), the new training data will not include this scaling and pruning of SalarySlab.

# Preprocessing: Create Buckets

### JobRole Bucketing

In [174]:
# Check for common averages of JobRole in Attrition
groupby_JobRole = remove_id_df[['JobRole', 'Attrition']].groupby('JobRole').mean().sort_values('Attrition')
groupby_JobRole

,Attrition
JobRole,
Research Director,0.025000
Manager,0.049020
Healthcare Representative,0.068702
Manufacturing Director,0.068966
Research Scientist,0.160959
Sales Executive,0.174847
Human Resources,0.230769
Laboratory Technician,0.239382
Sales Representative,0.397590


In [175]:
bucket_by_attrition_df = bucket_by_Attrition_Average(remove_id_df, 0.068, 0.069, 'JobRole', 'bucket_JobRole_0_068')

Number of unique values in 'JobRole' are: 9
Sales Executive              326
Research Scientist           292
Laboratory Technician        259
Manufacturing Director       145
Healthcare Representative    131
Manager                      102
Sales Representative          83
Research Director             80
Human Resources               52
Name: JobRole, dtype: int64

Bucket 'bucket_JobRole_0_068' includes the values: ['Healthcare Representative', 'Manufacturing Director']

Number of unique values in 'JobRole' are: 8
Sales Executive          326
Research Scientist       292
bucket_JobRole_0_068     276
Laboratory Technician    259
Manager                  102
Sales Representative      83
Research Director         80
Human Resources           52
Name: JobRole, dtype: int64


In [176]:
bucket_by_attrition_df = bucket_by_Attrition_Average(bucket_by_attrition_df, 0.16, 0.18, 'JobRole', 'bucket_JobRole_0_16')


Number of unique values in 'JobRole' are: 8
Sales Executive          326
Research Scientist       292
bucket_JobRole_0_068     276
Laboratory Technician    259
Manager                  102
Sales Representative      83
Research Director         80
Human Resources           52
Name: JobRole, dtype: int64

Bucket 'bucket_JobRole_0_16' includes the values: ['Research Scientist', 'Sales Executive']

Number of unique values in 'JobRole' are: 7
bucket_JobRole_0_16      618
bucket_JobRole_0_068     276
Laboratory Technician    259
Manager                  102
Sales Representative      83
Research Director         80
Human Resources           52
Name: JobRole, dtype: int64


In [177]:
bucket_by_attrition_df = bucket_by_Attrition_Average(bucket_by_attrition_df, 0.23, 0.24, 'JobRole', 'bucket_JobRole_0_23')


Number of unique values in 'JobRole' are: 7
bucket_JobRole_0_16      618
bucket_JobRole_0_068     276
Laboratory Technician    259
Manager                  102
Sales Representative      83
Research Director         80
Human Resources           52
Name: JobRole, dtype: int64

Bucket 'bucket_JobRole_0_23' includes the values: ['Human Resources', 'Laboratory Technician']

Number of unique values in 'JobRole' are: 6
bucket_JobRole_0_16     618
bucket_JobRole_0_23     311
bucket_JobRole_0_068    276
Manager                 102
Sales Representative     83
Research Director        80
Name: JobRole, dtype: int64


### EducationField Bucketing

In [178]:
# Check for common averages of EducationField in Attrition
groupby_EducationField = bucket_by_attrition_df[['EducationField', 'Attrition']].groupby('EducationField').mean().sort_values('Attrition')
groupby_EducationField

,Attrition
EducationField,
Other,0.134146
Medical,0.135776
Life Sciences,0.146865
Marketing,0.220126
Technical Degree,0.242424
Human Resources,0.259259


In [179]:
bucket_by_attrition_df = bucket_by_Attrition_Average(bucket_by_attrition_df, 0.13, 0.14, 'EducationField', 'bucket_EducationField_0_13')


Number of unique values in 'EducationField' are: 6
Life Sciences       606
Medical             464
Marketing           159
Technical Degree    132
Other                82
Human Resources      27
Name: EducationField, dtype: int64

Bucket 'bucket_EducationField_0_13' includes the values: ['Other', 'Medical']

Number of unique values in 'EducationField' are: 5
Life Sciences                 606
bucket_EducationField_0_13    546
Marketing                     159
Technical Degree              132
Human Resources                27
Name: EducationField, dtype: int64


In [180]:
bucket_by_attrition_df = bucket_by_Attrition_Average(bucket_by_attrition_df, 0.22, 0.25, 'EducationField', 'bucket_EducationField_0_22')


Number of unique values in 'EducationField' are: 5
Life Sciences                 606
bucket_EducationField_0_13    546
Marketing                     159
Technical Degree              132
Human Resources                27
Name: EducationField, dtype: int64

Bucket 'bucket_EducationField_0_22' includes the values: ['Marketing', 'Technical Degree']

Number of unique values in 'EducationField' are: 4
Life Sciences                 606
bucket_EducationField_0_13    546
bucket_EducationField_0_22    291
Human Resources                27
Name: EducationField, dtype: int64


### (Neural Network Model 9 was removed for being IQR variant and too fragile and irrelevant)

# Create Neural Network Model 10

In [181]:
start_threshold = 1000
end_threshold = 1001
max_epoch = 10
important_model_info = 'bucket JobRole and EducationField with no outliers removed'
model_name = 'Model 10'

columns = [['DailyRate', 'DistanceFromHome']]
for column in columns:
    df = bucket_by_attrition_df.copy()
    # break
    run_hyperparameters(f'{model_name}: {important_info}: try bucketing the average value of Attrition in association with a value of the \
columns of JobRole and EducationField (-- thereby, reducing the number of \
input/features) in order to reduce the learning complexity of the neural network;\
(these two columns were selected b/c they had the largest number of distinct values from amongst the\
the columns that were categorical instead of numberical); try WITH \
dropping EmpID and EmployeeNumber and using z-scores (not IQR) that are so high that \
NO outliers are found (so keeping all rows); \
"{column}" are \
the only columns with outliers removed; try hyperparameters across thresholds \
{start_threshold} to {end_threshold} excluding {end_threshold} with max_epoch {max_epoch}', 
                        df_with_preprocessing=df,
                        df_without_some_preprocessing_for_test_data=remove_id_df,
                        columns_to_remove_outliers_from=column, 
                        begin_threshold=start_threshold,
                        end_threshold=end_threshold,
                        column_to_remove='',
                        max_epochs=max_epoch,
                        restrict_columns_to_interquartile=False,
                        use_equal_number_outliers_as_signal_to_bypass_z_score_threshold_analysis=False,
                        run_model_against_X_test_scaled_with_outliers=False)

# Make a beep sound when code is finished running
winsound.Beep(freq, duration)

Trial 60 Complete [00h 00m 07s]
val_accuracy: 0.8396739363670349

Best val_accuracy So Far: 0.8396739363670349
Total elapsed time: 00h 07m 19s
12/12 - 0s - loss: nan - accuracy: 0.8397 - 183ms/epoch - 15ms/step
[{'model_accuracy_WITHOUT_outliers_in_test_data': 0.8396739363670349, 'timestamp': '20231129-032046', 'z_threshold': 1000, 'columns to remove outliers from': ['DailyRate', 'DistanceFromHome'], 'model_accuracy_WITH_outliers_in_test_data': 'did not run with outliers included', 'number of columns': 57, 'param': {'num_layers': 1, 'activation_layer_input': 'leaky_relu', 'neurons_layer_input': 66, 'learning_rate': 0.0002965488058019691, 'tuner/epochs': 2, 'tuner/initial_epoch': 0, 'tuner/bracket': 2, 'tuner/round': 0}, 'columns': Index(['Age', 'Attrition', 'DailyRate', 'DistanceFromHome', 'Education',
       'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement', 'JobLevel',
       'JobSatisfaction', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',
       'PercentSalaryHike', 

### Result: Neural Network Model 10
Neural Network Model 10 had an accuracy of 83.967% with 1470 rows out 1470 rows with creating 5 buckets and thereby reducing the number of features to 57 from 62 (and so complexity). (This was not tested against a dataset 'with outliers included' b/c there were no outliers-- all 1470 rows are present and accounted for).

# Preprocessing: Convert Overtime to '1's and '0's
From other analyses in HR_r_squared.ipynb, it was found that 'Overtime' had the highest r-squared value (6%) in explaining the variation in 'Attrition', and so, it was worth investigating z-threshold series on it after it's values have been changed from 'yes' and 'no' to '1' and '0', respectively.

In [182]:
overtime1And0_df = bucket_by_attrition_df.copy()

overtime1And0_df['OverTime'].unique()

array(['No', 'Yes'], dtype=object)

In [183]:
print(overtime1And0_df.shape)
print(overtime1And0_df.info())
overtime1And0_df.head()

(1470, 33)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 33 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       1470 non-null   int64  
 1   AgeGroup                  1470 non-null   object 
 2   Attrition                 1470 non-null   int64  
 3   BusinessTravel            1470 non-null   object 
 4   DailyRate                 1470 non-null   int64  
 5   Department                1470 non-null   object 
 6   DistanceFromHome          1470 non-null   int64  
 7   Education                 1470 non-null   int64  
 8   EducationField            1470 non-null   object 
 9   EnvironmentSatisfaction   1470 non-null   int64  
 10  Gender                    1470 non-null   object 
 11  HourlyRate                1470 non-null   int64  
 12  JobInvolvement            1470 non-null   int64  
 13  JobLevel                  1470 non-null   int64  
 1

,Age,AgeGroup,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,SalarySlab,MonthlyRate,NumCompaniesWorked,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,18,18-25,1,Travel_Rarely,230,Research & Development,3,3,Life Sciences,3,Male,54,3,1,bucket_JobRole_0_23,3,Single,1420,Upto 5k,25233,1,No,13,3,3,0,0,2,3,0,0,0,0.0
1,18,18-25,0,Travel_Rarely,812,Sales,10,3,bucket_EducationField_0_13,4,Female,69,2,1,Sales Representative,3,Single,1200,Upto 5k,9724,1,No,12,3,1,0,0,2,3,0,0,0,0.0
2,18,18-25,1,Travel_Frequently,1306,Sales,5,3,bucket_EducationField_0_22,2,Male,69,3,1,Sales Representative,2,Single,1878,Upto 5k,8059,1,Yes,14,3,4,0,0,3,3,0,0,0,0.0
3,18,18-25,0,Non-Travel,287,Research & Development,5,2,Life Sciences,2,Male,73,3,1,bucket_JobRole_0_16,4,Single,1051,Upto 5k,13493,1,No,15,3,4,0,0,2,3,0,0,0,0.0
4,18,18-25,1,Non-Travel,247,Research & Development,8,1,bucket_EducationField_0_13,3,Male,80,3,1,bucket_JobRole_0_23,3,Single,1904,Upto 5k,13556,1,No,12,3,4,0,0,0,3,0,0,0,0.0


In [184]:
overtime1And0_df['OverTime'] = overtime1And0_df['OverTime'].replace('No',"0")
overtime1And0_df['OverTime'] = overtime1And0_df['OverTime'].replace('Yes',"1")
overtime1And0_df['OverTime'] = overtime1And0_df['OverTime'].astype('int64')
print(overtime1And0_df.shape)
print(overtime1And0_df.info())
overtime1And0_df.head()

(1470, 33)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 33 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       1470 non-null   int64  
 1   AgeGroup                  1470 non-null   object 
 2   Attrition                 1470 non-null   int64  
 3   BusinessTravel            1470 non-null   object 
 4   DailyRate                 1470 non-null   int64  
 5   Department                1470 non-null   object 
 6   DistanceFromHome          1470 non-null   int64  
 7   Education                 1470 non-null   int64  
 8   EducationField            1470 non-null   object 
 9   EnvironmentSatisfaction   1470 non-null   int64  
 10  Gender                    1470 non-null   object 
 11  HourlyRate                1470 non-null   int64  
 12  JobInvolvement            1470 non-null   int64  
 13  JobLevel                  1470 non-null   int64  
 1

,Age,AgeGroup,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,SalarySlab,MonthlyRate,NumCompaniesWorked,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,18,18-25,1,Travel_Rarely,230,Research & Development,3,3,Life Sciences,3,Male,54,3,1,bucket_JobRole_0_23,3,Single,1420,Upto 5k,25233,1,0,13,3,3,0,0,2,3,0,0,0,0.0
1,18,18-25,0,Travel_Rarely,812,Sales,10,3,bucket_EducationField_0_13,4,Female,69,2,1,Sales Representative,3,Single,1200,Upto 5k,9724,1,0,12,3,1,0,0,2,3,0,0,0,0.0
2,18,18-25,1,Travel_Frequently,1306,Sales,5,3,bucket_EducationField_0_22,2,Male,69,3,1,Sales Representative,2,Single,1878,Upto 5k,8059,1,1,14,3,4,0,0,3,3,0,0,0,0.0
3,18,18-25,0,Non-Travel,287,Research & Development,5,2,Life Sciences,2,Male,73,3,1,bucket_JobRole_0_16,4,Single,1051,Upto 5k,13493,1,0,15,3,4,0,0,2,3,0,0,0,0.0
4,18,18-25,1,Non-Travel,247,Research & Development,8,1,bucket_EducationField_0_13,3,Male,80,3,1,bucket_JobRole_0_23,3,Single,1904,Upto 5k,13556,1,0,12,3,4,0,0,0,3,0,0,0,0.0


# Create Neural Network Model 11.Z-Series
From other analyses in HR_r_squared.ipynb, it was found that 'Overtime' had the highest r-squared value (6%) in explaining the variation in 'Attrition', and so, it was worth investigating z-threshold series on it after it's values had been changed from 'yes' and 'no' to '1' and '0', respectively.

In [187]:
# Create Neural Network Model with dropped columns EmpID and EmployeeNumber and do NOT use 1.5 * IQR Rule
start_threshold = 1
end_threshold = 11
max_epoch = 10
model_name = 'Model 11'
important_info = 'Overtime: And bucketed'

columns = ['OverTime']
for column in columns:
    df = overtime1And0_df.copy()
    # break
    run_hyperparameters(optimization_title=f'{model_name}: {important_info}: try WITH dropping EmpID and EmployeeNumber and using z-scores;\
"{column}" are the only columns with outliers removed; try \
hyperparameters across thresholds {start_threshold} to {end_threshold} excluding \
{end_threshold} with max_epoch {max_epoch}',
                        df_with_preprocessing=df,
                        df_without_some_preprocessing_for_test_data=df,
                        columns_to_remove_outliers_from=[column],
                        begin_threshold=start_threshold,
                        end_threshold=end_threshold,
                        column_to_remove='',
                        max_epochs=max_epoch,
                        restrict_columns_to_interquartile=False,
                        use_equal_number_outliers_as_signal_to_bypass_z_score_threshold_analysis=True,
                        run_model_against_X_test_scaled_with_outliers=True)

# Make a beep sound when code is finished running
winsound.Beep(freq, duration)

Trial 60 Complete [00h 00m 08s]
val_accuracy: 0.8396739363670349

Best val_accuracy So Far: 0.8396739363670349
Total elapsed time: 00h 07m 41s
12/12 - 0s - loss: nan - accuracy: 0.8397 - 118ms/epoch - 10ms/step
12/12 - 0s - loss: nan - accuracy: 0.8397 - 23ms/epoch - 2ms/step
['OverTime']
<class 'list'>
Outlier rows:
Int64Index([], dtype='int64')
0
['OverTime']
<class 'list'>
Outlier rows:
Int64Index([], dtype='int64')
0
['OverTime']
<class 'list'>
Outlier rows:
Int64Index([], dtype='int64')
0
['OverTime']
<class 'list'>
Outlier rows:
Int64Index([], dtype='int64')
0
['OverTime']
<class 'list'>
Outlier rows:
Int64Index([], dtype='int64')
0
['OverTime']
<class 'list'>
Outlier rows:
Int64Index([], dtype='int64')
0
['OverTime']
<class 'list'>
Outlier rows:
Int64Index([], dtype='int64')
0
['OverTime']
<class 'list'>
Outlier rows:
Int64Index([], dtype='int64')
0
[{'model_accuracy_WITHOUT_outliers_in_test_data': 0.8396739363670349, 'timestamp': '20231129-032046', 'z_threshold': 1000, 'columns

### Result: Neural Network Model 11
Neural Network Model11 had an accuracy of 89.394% with 1054 rows out 1470 rows. When the model is run against all the data 1470 out of 1470 rows it gets 83.967% which is standard.

# Reproduce Results of Model Found By Hyperband

## Here are the specs from the best fitting model

#### (These specifications along many other optimization tests were captured in the file ./DataFiles/optimization_results.txt)<br><br>

{'model_accuracy_WITHOUT_outliers_in_test_data': 0.8939393758773804<br>
'timestamp': 20231129-041352<br>
'z_threshold': 1<br>
'columns to remove outliers from': ['OverTime']<br>
'model_accuracy_WITH_outliers_in_test_data': 0.8396739363670349<br>
'number of columns': 56<br>
'param': {<br>
      &nbsp;&nbsp;&nbsp;&nbsp;"num_layers": 1, <br>
      &nbsp;&nbsp;&nbsp;&nbsp;"activation_layer_input": "leaky_relu", <br>
      &nbsp;&nbsp;&nbsp;&nbsp;"neurons_layer_input": 65, <br>
      &nbsp;&nbsp;&nbsp;&nbsp;"learning_rate": 0.0002965488058019691, <br>
      &nbsp;&nbsp;&nbsp;&nbsp;"tuner/epochs": 2, <br>
      &nbsp;&nbsp;&nbsp;&nbsp;"tuner/initial_epoch": 0, <br>
      &nbsp;&nbsp;&nbsp;&nbsp;"tuner/bracket": 2, <br>
      &nbsp;&nbsp;&nbsp;&nbsp;"tuner/round": 0<br>
      }<br>
'columns': Index(['Age', 'Attrition', 'DailyRate', 'DistanceFromHome', 'Education',<br>
       'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement', 'JobLevel',<br>
       'JobSatisfaction', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',<br>
       'OverTime', 'PercentSalaryHike', 'PerformanceRating',<br>
       'RelationshipSatisfaction', 'StockOptionLevel', 'TotalWorkingYears',<br>
       'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany',<br>
       'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager',<br>
       'AgeGroup_18-25', 'AgeGroup_26-35', 'AgeGroup_36-45', 'AgeGroup_46-55',<br>
       'AgeGroup_55+', 'BusinessTravel_Non-Travel',<br>
       'BusinessTravel_TravelRarely', 'BusinessTravel_Travel_Frequently',<br>
       'BusinessTravel_Travel_Rarely', 'Department_Human Resources',<br>
       'Department_Research & Development', 'Department_Sales',<br>
       'EducationField_Human Resources', 'EducationField_Life Sciences',<br>
       'EducationField_bucket_EducationField_0_13',<br>
       'EducationField_bucket_EducationField_0_22', 'Gender_Female',<br>
       'Gender_Male', 'JobRole_Manager', 'JobRole_Research Director',<br>
       'JobRole_Sales Representative', 'JobRole_bucket_JobRole_0_068',<br>
       'JobRole_bucket_JobRole_0_16', 'JobRole_bucket_JobRole_0_23',<br>
       'MaritalStatus_Divorced', 'MaritalStatus_Married',<br>
       'MaritalStatus_Single', 'SalarySlab_10k-15k', 'SalarySlab_15k+',<br>
       'SalarySlab_5k-10k', 'SalarySlab_Upto 5k'],<br>
      dtype='object')<br>
'removed column': <br>
'restrict columns to 1.5 * IQR': False<br>
'number of outlier rows': 416<br>
'number of rows': 1054<br>
'max_epoch': 10<br>
'hyperband seed': 1<br>
'np.rand.seed and tf.random.set_seed': 42<br>
'Optimization Title': Model 11: Overtime: And bucketed: try WITH dropping EmpID and EmployeeNumber and using z-scores;"OverTime" are <br>the only columns with outliers removed; try hyperparameters across thresholds 1 to 11 excluding 11 with max_epoch 10<br>
}<br>

### Test Best Model Against All 1470 Rows

In [193]:
from tensorflow.keras.models import load_model
# Replace 'path_to_my_model.h5' with the actual path to your .h5 file
model11 = load_model('./DataFiles/model_20231129-050319_89.39.h5')


X, y = extract_X_y(pd.get_dummies(overtime1And0_df), 'Attrition')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=3,
                                                    stratify=y)

# Standardize X_train and X_test
X_train_scaled, X_test_scaled = scale_it(X_train, X_test)

# For evaluating the model
loss, accuracy = model11.evaluate(X_test_scaled, y_test)

# Print with more decimal places
print("Loss: {:.5f}, Accuracy: {:.5f}".format(loss, accuracy))


12/12 [==============================] - 0s 909us/step - loss: nan - accuracy: 0.8397
Loss: nan, Accuracy: 0.83967


### Test Best Model Against Z-Score Of 1 Restricted Data

In [196]:
#  Test with a z-score-threshold of 1

# Get Df without outliers
columns_to_remove_outliers_from = ['OverTime']
z_threshold = 1
df = overtime1And0_df
outlier_rows, no_outliers_df = remove_outliers(df, z_threshold, columns_to_remove_outliers_from)

# Extract X and y
X, y = extract_X_y(pd.get_dummies(no_outliers_df), 'Attrition')

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=3,
                                                    stratify=y)

# Standardize X_train and X_test
X_train_scaled, X_test_scaled = scale_it(X_train, X_test)

# For evaluating the model
loss, accuracy = model11.evaluate(X_test_scaled, y_test)

# Print with more decimal places
print("Loss: {:.5f}, Accuracy: {:.5f}".format(loss, accuracy))


['OverTime']
<class 'list'>
Outlier rows:
Int64Index([   2,    6,    8,   13,   15,   16,   17,   19,   22,   23,
            ...
            1446, 1447, 1450, 1451, 1453, 1454, 1455, 1461, 1462, 1468],
           dtype='int64', length=416)
9/9 [==============================] - 0s 7ms/step - loss: nan - accuracy: 0.8939
Loss: nan, Accuracy: 0.89394


# Preliminary Conclusion For Neural Network Models<sup>1</sup>

After creating and analyzing nearly 100 models (including the models from the Z-Series), the Neural Network Model11 shows the most promise of the Neural Networks because it has an accuracy of 89.394% on 1054 out 1470 rows (z-score-threshold of 1 removes 416 rows) and 83.967% on 1470 out 1470 rows. This model has 1 layer (including the input layer but excluding the output layer) of 65 neurons and uses 'leaky_relu' as its activation for this layer, 'sigmoid' for the output layer, and a learning rate of '0.0002965' with an 'Adam' Optimizer.

<sup>1</sup><sub style='font-size: 10;'>The Neural Network Models will be compared with the Random Forest Classifier, Support Vector, and Logistic Regression Models next.</sub>

# Final Conclusion

The Support Vector Model2 had the highest accuracy of 88.587% against all 1470 rows compared to Logistic Model3's 88.315%, Support Vector Model3's 88.043%, and Random Forest Classifier Model3's 85.870%, and Nerual Network Model11's 83.967%. However, when tested against z-score 1 restricted data (in reference to the Overtime column).<sup>2</sup> The Logistic Regression Model3 had the highest accuracy of 90.152 against 1054 rows out of 1470 rows<sup>3</sup> as compared to Support Vector Model3's 89.394%, Random Forest Classifier Model3's 89.394%, and Neural Network Model's 89.394%. <br>

So, in summary, of the 100+ models considered<sup>4</sup>, the Logistic Regression Model3 and Support Vector Model3 are nearly identical in performance are tied for the best all around models.<br>


<sup>2</sup><sub style='font-size: 10;'> Support Vector Model2 was NOT tested against z-score-1-outliers-removed data because it has an incompatible number of features (62 features as opposed to 61) that's why Support Vector Model3 was created and used.</sub>

<sup>3</sup><sub style='font-size: 10;'> The z-score-1-outliers-removed data reduces the number of rows to 1054 rows out of 1470 rows.</sub>

<sup>4</sup><sub style='font-size: 10;'> 98 Neural Network Models, 3 Random Forest Classifier Models, 3 Support Vector Models, and 3 Logistic Regression Models.</sub>